Una vez que tienes un modelo hermoso que hace predicciones increíbles, ¿qué haces con él? Bueno, ¡tienes que ponerlo en producción! Esto podría ser tan simple como ejecutar el modelo en un lote de datos, y tal vez escribir un script que ejecute este modelo todas las noches. Sin embargo, a menudo es mucho más complicado. Varias partes de su infraestructura pueden necesitar usar este modelo en datos en vivo, en cuyo caso probablemente querrá envolver su modelo en un servicio web: de esta manera, cualquier parte de su infraestructura puede consultar el modelo en cualquier momento utilizando una simple API REST (o algún otro protocolo), como discutimos en el Capítulo 2. Pero a medida que pase el tiempo, tendrás que volver a entrenar regularmente tu modelo con datos nuevos y llevar la versión actualizada a la producción. Debe manejar el control de versiones del modelo, hacer la transición con gracia de un modelo al siguiente, posiblemente volver al modelo anterior en caso de problemas, y tal vez ejecutar varios modelos diferentes en paralelo para realizar experimentos A/B.⁠1 Si su producto tiene éxito, su servicio puede comenzar a recibir un gran número de consultas por segundo (QPS), y debe escalar para soportar la carga. Una gran solución para ampliar su servicio, como verá en este capítulo, es usar TF Serving, ya sea en su propia infraestructura de hardware o a través de un servicio en la nube como Google Vertex AI.⁠2 Se encargará de servir de manera eficiente a su modelo, manejar transiciones de modelo elegantes y más. Si utiliza la plataforma en la nube, también obtendrá muchas características adicionales, como potentes herramientas de monitoreo.

Además, si tiene muchos datos de entrenamiento y modelos de computación intensiva, el tiempo de entrenamiento puede ser prohibitivamente largo. Si su producto necesita adaptarse a los cambios rápidamente, entonces un largo tiempo de entrenamiento puede ser un espectáculo (por ejemplo, piense en un sistema de recomendación de noticias que promueva las noticias de la semana pasada). Tal vez lo que es aún más importante, un largo tiempo de entrenamiento te impedirá experimentar con nuevas ideas. En el aprendizaje automático (como en muchos otros campos), es difícil saber de antemano qué ideas funcionarán, por lo que debe probar tantas como sea posible, lo más rápido posible. Una forma de acelerar el entrenamiento es usar aceleradores de hardware como GPU o TPU. Para ir aún más rápido, puedes entrenar un modelo en varias máquinas, cada una equipada con múltiples aceleradores de hardware. La sencilla pero potente API de estrategias de distribución de TensorFlow lo hace fácil, como verás.

En este capítulo veremos cómo implementar modelos, primero usando TF Serving y luego usando Vertex AI. También echaremos un vistazo rápido a la implementación de modelos en aplicaciones móviles, dispositivos integrados y aplicaciones web. Luego discutiremos cómo acelerar los cálculos utilizando GPU y cómo entrenar modelos en múltiples dispositivos y servidores utilizando la API de estrategias de distribución. Por último, exploraremos cómo entrenar modelos y afinar sus hiperparámetros a escala utilizando Vertex AI. Esos son muchos temas para discutir, ¡así que vamos a sumergirnos!


# Sirviendo a un modelo TensorFlow

Una vez que hayas entrenado un modelo de TensorFlow, puedes usarlo fácilmente en cualquier código de Python: si es un modelo de Keras, ¡solo tienes que llamar al método itspredictpredict()! Pero a medida que su infraestructura crece, llega un punto en el que es preferible envolver su modelo en un pequeño servicio cuyo único papel es hacer predicciones y hacer que el resto de la infraestructura lo consulte (por ejemplo, a través de una API REST o gRPC). Esto desacopla su modelo del resto de la infraestructura, lo que permite cambiar fácilmente las versiones del modelo o escalar el servicio según sea necesario (independientemente del resto de su infraestructura), realizar experimentos A/B y asegurarse de que todos sus componentes de software dependan de las mismas versiones del modelo. También simplifica las pruebas y el desarrollo, y más. Podrías crear tu propio microservicio usando cualquier tecnología que quieras (por ejemplo, usando la biblioteca Flask), pero ¿por qué reinventar la rueda cuando solo puedes usar TF Serving?

## Uso de la porción de TensorFlow

TF Serving es un servidor de modelos muy eficiente y probado en batalla, escrito en C++. Puede soportar una carga alta, servir a múltiples versiones de sus modelos y ver un repositorio de modelos para implementar automáticamente las últimas versiones y más (ver Figura 19-1).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1901.png)

(_Figura 19-1. TF Serving puede servir a varios modelos e implementar automáticamente la última versión de cada modelo_)

Así que supongamos que has entrenado un modelo MNIST usando Keras, y quieres implementarlo en TF Serving. Lo primero que tienes que hacer es exportar este modelo al formato SavedModel, introducido en el capítulo 10.

### Exportación de modelos guardados

Ya sabes cómo guardar el modelo: solo llama a model.save(). Ahora, para ver la versión del modelo, solo tienes que crear un subdirectorio para cada versión del modelo. ¡Fácil!

In [ ]:
from pathlib import Path
import tensorflow as tf

X_train, X_valid, X_test = [...]  # load and split the MNIST dataset
model = [...]  # build & train an MNIST model (also handles image preprocessing)

model_name = "my_mnist_model"
model_version = "0001"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

Por lo general, es una buena idea incluir todas las capas de preprocesamiento en el modelo final que exporta para que pueda ingerir datos en su forma natural una vez que se implementen en producción. Esto evita tener que encargarse del preprocesamiento por separado dentro de la aplicación que utiliza el modelo. La agrupación de los pasos de preprocesamiento dentro del modelo también hace que sea más fácil actualizarlos más adelante y limita el riesgo de desajuste entre un modelo y los pasos de preprocesamiento que requiere.

#### ADVERTENCIA

Dado que un SavedModel guarda el gráfico de cálculo, solo se puede utilizar con modelos que se basan exclusivamente en operaciones de TensorFlow, excluyendo la operación `tf.py_function()`, que envuelve código Python arbitrario.

#### ------------------------------------------------------------------------------------------------------------

TensorFlow viene con una pequeña interfaz de línea de comandos `save_model_cli` para inspeccionar SavedModels. Utilicémoslo para inspeccionar nuestro modelo exportado:

In [ ]:
$ saved_model_cli show --dir my_mnist_model/0001The given SavedModel contains the following tag-sets:
'serve'

¿Qué significa este resultado? Bueno, un SavedModel contiene uno o más metágrafos. Un metágrafo es un gráfico de cálculo más algunas definiciones de firma de funciones, incluidos sus nombres, tipos y formas de entrada y salida. Cada metágrafo se identifica mediante un conjunto de etiquetas. Por ejemplo, es posible que desee tener un metágrafo que contenga el gráfico de cálculo completo, incluidas las operaciones de entrenamiento: normalmente lo etiquetaría como "train" Y es posible que tenga otro metágrafo que contenga un gráfico de cálculo podado con solo las operaciones de predicción, incluidas algunas operaciones específicas de la GPU: este podría estar etiquetado como `"serve"`, `"gpu"` Es posible que también quieras tener otros metágrafos. Esto se puede hacer utilizando la API SavedModel de bajo nivel de TensorFlow. Sin embargo, cuando se guarda un modelo de Keras utilizando el método itssave `save()`, se guarda un solo metágrafo etiquetado como `"serve"` Inspeccionemos este conjunto de etiquetas "serve":

In [ ]:
$ saved_model_cli show --dir 0001my_mnist_model --tag_set serveThe given SavedModel MetaGraphDef contains SignatureDefs with these keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"

Este metagrama contiene dos definiciones de firma: una función de inicialización llamada `"__saved_model_init_op"`, de la que no necesita preocuparse, y una función de servicio predeterminada llamada "serving_default". Al guardar un modelo de Keras, la función de publicación predeterminada es el método `call()` del modelo, que hace predicciones, como ya sabe. Obtengamos más detalles sobre esta función de publicación:

In [ ]:
$ saved_model_cli show --dir 0001my_mnist_model --tag_set serve\--signature_def serving_defaultThe given SavedModel SignatureDef contains the following input(s):
  inputs['flatten_input'] tensor_info:
      dtype: DT_UINT8
      shape: (-1, 28, 28)
      name: serving_default_flatten_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['dense_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 10)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict

Tenga en cuenta que la entrada de la función se denomina `"flatten_input"` y la salida se denomina `"dense_1"`. Estos corresponden a los nombres de las capas de entrada y salida del modelo Keras. También puede ver el tipo y la forma de los datos de entrada y salida. ¡Se ve bien!

Ahora que tienes un SavedModel, el siguiente paso es instalar TF Serving.

### Instalación e inicio del servicio de TensorFlow

Hay muchas formas de instalar TF Serving: usando el administrador de paquetes del sistema, usando una imagen de Docker⁠, instalando desde el código fuente y más. Dado que Colab se ejecuta en Ubuntu, podemos usar el administrador de paquetes apt de Ubuntu de esta manera:

In [ ]:
url = "https://storage.googleapis.com/tensorflow-serving-apt"
src = "stable tensorflow-model-server tensorflow-model-server-universal"
!echo 'deb {url} {src}'> /etc/apt/sources.list.d/tensorflow-serving.list!curl '{url}/tensorflow-serving.release.pub.gpg' | apt-key add -
!apt update -q && apt-get install -y tensorflow-model-server
%pipinstall -q -U tensorflow-serving-api

Este código comienza agregando el repositorio de paquetes de TensorFlow a la lista de fuentes de paquetes de Ubuntu. Luego descarga la clave GPG pública de TensorFlow y la agrega a la lista de claves del administrador de paquetes para que pueda verificar las firmas de los paquetes de TensorFlow. A continuación, utiliza apt para instalar el paquete `tensorflow-model-server`. Por último, instala la biblioteca `tensorflow-serving-api`, que necesitaremos para comunicarnos con el servidor.

Ahora queremos iniciar el servidor. El comando requerirá la ruta absoluta del directorio del modelo base (es decir, la ruta a `my_mnist_model`, no `0001`), así que guardémosla en la variable de entorno `MODEL_DIR`:

In [ ]:
import os

os.environ["MODEL_DIR"] = str(model_path.parent.absolute())

Luego podemos iniciar el servidor:

%%bash--bg tensorflow_model_server \ --port=8500 \ --rest_api_port=8501 \ --model_name=my_mnist_model \ --model_base_path="${MODEL_DIR}" >my_server.log 2>&1

En Jupyter o Colab, el comando mágico `%%bash --bg` ejecuta la celda como un script bash, ejecutándolo en segundo plano. La parte `>my_server.log 2>&1` redirige la salida estándar y el error estándar al archivo my_server.log. ¡Y eso es! TF Serving ahora se ejecuta en segundo plano y sus registros se guardan en my_server.log. Cargó nuestro modelo MNIST (versión 1) y ahora está esperando solicitudes de gRPC y REST, respectivamente, en los puertos 8500 y 8501.

#### EJECUTANDO EL SERVICIO DE TF EN UN CONTENEDOR DE DOCKER

Si está ejecutando la computadora portátil en su propia máquina y ha instalado Docker, puede ejecutar `docker pull tensorflow/serving` en una terminal para descargar la imagen de TF Serving. El equipo de TensorFlow recomienda encarecidamente este método de instalación porque es simple, no afectará su sistema y ofrece un alto rendimiento.⁠ Para iniciar el servidor dentro de un contenedor Docker, puede ejecutar el siguiente comando en una terminal:

In [ ]:
$ docker run -it --rm -v"/path/to/my_mnist_model:/models/my_mnist_model" \-p 8500:8500 -p 85018501 -e MODEL_NAME=my_mnist_model tensorflow/serving

Esto es lo que significan todas estas opciones de línea de comandos:

`-it`

    Hace que el contenedor sea interactivo (para que puedas presionar Ctrl-C para detenerlo) y muestra la salida del servidor.

`--rm`

    Elimina el contenedor cuando lo detienes: no hay necesidad de abarrotar tu máquina con contenedores interrumpidos. Sin embargo, no elimina la imagen.

`-v "/path/to/my_mnist_model:/models/my_mnist_model"`

    Hace que el directorio my_mnist_model del host esté disponible para el contenedor en la ruta /models/mnist_model. Debe reemplazar /path/to/my_mnist_model con la ruta absoluta de este directorio. En Windows, recuerde usar \ en lugar de / en la ruta del host, pero no en la ruta del contenedor (ya que el contenedor se ejecuta en Linux).

`-p 8500:8500`

    Hace que el motor Docker reenvíe el puerto TCP 8500 del host al puerto TCP 8500 del contenedor. De forma predeterminada, TF Serving utiliza este puerto para servir la API de gRPC.

`-p 8501:8501`

    Reenvía el puerto TCP 8501 del host al puerto TCP 8501 del contenedor. La imagen de Docker está configurada para usar este puerto de forma predeterminada para servir a la API REST.

`-e MODEL_NAME=my_mnist_model`

    Establece la variable de entorno `MODEL_NAME` del contenedor, para que TF Serving sepa qué modelo servir. De forma predeterminada, buscará modelos en el directorio /models y automáticamente ofrecerá la última versión que encuentre.

`tensorflow/serving`

    Este es el nombre de la imagen que se va a ejecutar.
    
#### ------------------------------------------------------------------------------------------------------------

Ahora que el servidor está en funcionamiento, vamos a consultarlo, primero usando la API REST y luego la API gRPC.


### Consultando el servicio de TF a través de la API REST

Comenzamos creando la consulta. Debe contener el nombre de la firma de la función a la que desea llamar y, por supuesto, los datos de entrada. Dado que la solicitud debe usar el formato JSON, tenemos que convertir las imágenes de entrada de una matriz NumPy a una lista de Python:

In [ ]:
import json

X_new = X_test[:3]  # pretend we have 3 new digit images to classify
request_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

Tenga en cuenta que el formato JSON está 100% basado en texto. La cadena de solicitud se ve así:

In [ ]:
request_json

#'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, ... ]]]}'

Ahora enviemos esta solicitud a TF Serving a través de una solicitud HTTP POST. Esto se puede hacer usando la biblioteca de `requests` (no forma parte de la biblioteca estándar de Python, pero está preinstalada en Colab):

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data=request_json)
response.raise_for_status()  # raise an exception in case of error
response = response.json()

Si todo va bien, la respuesta debería ser un diccionario que contenga una sola clave de `"predictions"` El valor correspondiente es la lista de predicciones. Esta lista es una lista de Python, así que convirtámosla en una matriz NumPy y redondeemos los flotantes que contiene al segundo decimal:

In [ ]:
import numpy as np
y_proba = np.array(response["predictions"])
y_proba.round(2)

#array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
#       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
#       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

¡Hurra, tenemos las predicciones! El modelo está cerca del 100 % seguro de que la primera imagen es un 7, 99 % seguro de que la segunda imagen es un 2, y el 97 % confía en que la tercera imagen es un 1. Eso es correcto.

La API REST es agradable y sencilla, y funciona bien cuando los datos de entrada y salida no son demasiado grandes. Además, casi cualquier aplicación cliente puede hacer consultas REST sin dependencias adicionales, mientras que otros protocolos no siempre están tan fácilmente disponibles. Sin embargo, se basa en JSON, que está basado en texto y es bastante detallado. Por ejemplo, tuvimos que convertir la matriz NumPy en una lista de Python, y cada flotante terminó representada como una cadena. Esto es muy ineficiente, tanto en términos de tiempo de serialización/deserialización (tenemos que convertir todos los flotadores en cadenas y vuelta) como en términos de tamaño de la carga útil: ¡muchos flotadores terminan siendo representados usando más de 15 caracteres, lo que se traduce en más de 120 bits para los flotadores de 32 bits! Esto resultará en una alta latencia y uso de ancho de banda al transferir grandes matrices NumPy. ⁠6 Por lo tanto, veamos cómo usar gRPC en su lugar.

#### TIP

Al transferir grandes cantidades de datos, o cuando la latencia es importante, es mucho mejor usar la API gRPC, si el cliente la admite, ya que utiliza un formato binario compacto y un protocolo de comunicación eficiente basado en el marco HTTP/2.

#### ---------------------------------------------------------------------------------------------------------------

### Consultando el servicio de TF a través de la API de gRPC

La API gRPC espera un búfer de protocolo `PredictRequest` serializado como entrada y genera un búfer de protocolo PredictResponse serializado. Estos protobufs son parte de la biblioteca `tensorflow-serving-api`, que instalamos anteriormente. Primero, creemos la solicitud:

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]  # == "flatten_input"
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

Este código crea un búfer de protocolo `PredictRequest` y completa los campos obligatorios, incluido el nombre del modelo (definido anteriormente), el nombre de la firma de la función que queremos llamar y, finalmente, los datos de entrada, en forma de un búfer de protocolo Tensor. La función `tf.make_tensor_proto()` crea un búfer de protocolo Tensor basado en el tensor o matriz NumPy dado, en este caso `X_new`.

A continuación, enviaremos la solicitud al servidor y obtendremos su respuesta. Para esto, necesitaremos la biblioteca grpcio, que está preinstalada en Colab:

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('localhost:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

El código es bastante sencillo: después de las importaciones, creamos un canal de comunicación gRPC a localhost en el puerto TCP 8500, luego creamos un servicio gRPC a través de este canal y lo usamos para enviar una solicitud, con un tiempo de espera de 10 segundos. Tenga en cuenta que la llamada es sincrónica: se bloqueará hasta que reciba la respuesta o cuando expire el período de tiempo de espera. En este ejemplo, el canal no es seguro (sin cifrado, sin autenticación), pero gRPC y TF Serving también admiten canales seguros a través de SSL/TLS.

Después, vamos a convertir el buffer de protocolo `PredictResponse` en un tensor:

In [ ]:
output_name = model.output_names[0]  # == "dense_1"
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

Si ejecuta este código e imprime `y_proba.round(2)`, obtendrá exactamente las mismas probabilidades de clase estimadas que antes. Y eso es todo: en solo unas pocas líneas de código, ahora puede acceder a su modelo de TensorFlow de forma remota, utilizando REST o gRPC.


### Implementación de una nueva versión de modelo

Ahora vamos a crear una nueva versión de modelo y exportar un SavedModel, esta vez al directorio my_mnist_model/0002:

In [ ]:
model = [...]  # build and train a new MNIST model version

model_version = "0002"
model_path = Path(model_name) / model_version
model.save(model_path, save_format="tf")

A intervalos regulares (el retraso es configurable), TF Serving comprueba el directorio de modelos para las nuevas versiones de los modelos. Si encuentra uno, maneja automáticamente la transición con gracia: de forma predeterminada, responde a las solicitudes pendientes (si las hay) con la versión del modelo anterior, mientras maneja las nuevas solicitudes con la nueva versión. Tan pronto como se haya respondido a todas las solicitudes pendientes, se descarga la versión del modelo anterior. Puedes ver esto en el trabajo en los registros de servicio de TF (en my_server.log):


In [ ]:
[...] Lectura del modelo guardado de: /models/my_mnist_model/0002 Lectura de metagráfico con etiquetas { serve } [...] Versión servible cargada correctamente {nombre: versión my_mnist_model: 2} Versión de servicio de quiescing {nombre: versión my_mnist_model: 1} Terminado la versión servible de quiescing {nombre: versión my_mnist_model: 1} Descargando la versión servible {nombre: versión my_mnist_model: 1}

#### TIP

Si el SavedModel contiene algunas instancias de ejemplo en el directorio de activos/extra, puede configurar TF Serving para ejecutar el nuevo modelo en estas instancias antes de comenzar a usarlo para servir solicitudes. Esto se llama calentamiento del modelo: se asegurará de que todo esté cargado correctamente, evitando largos tiempos de respuesta para las primeras solicitudes.

#### ---------------------------------------------------------------------------------------------------------

Este enfoque ofrece una transición suave, pero puede usar demasiada RAM, especialmente la RAM de la GPU, que generalmente es la más limitada. En este caso, puede configurar TF Serving para que maneje todas las solicitudes pendientes con la versión del modelo anterior y la descargue antes de cargar y usar la versión del nuevo modelo. Esta configuración evitará tener dos versiones de modelo cargadas al mismo tiempo, pero el servicio no estará disponible durante un corto período.

Como puede ver, TF Serving hace que sea sencillo implementar nuevos modelos. Además, si descubres que la versión 2 no funciona tan bien como esperabas, volver a la versión 1 es tan simple como eliminar el directorio my_mnist_model/0002.

#### TIP

Otra gran característica de TF Serving es su capacidad de procesamiento por lotes automático, que puede activar usando la opción `--enable_batching` al inicio. Cuando TF Serving recibe varias solicitudes en un corto período de tiempo (el retraso es configurable), las agrupará automáticamente antes de usar el modelo. Esto ofrece un aumento significativo del rendimiento al aprovechar la potencia de la GPU. Una vez que el modelo devuelve las predicciones, TF Serving envía cada predicción al cliente correcto. Puede cambiar un poco de latencia por un mayor rendimiento aumentando el retraso del procesamiento por lotes (consulte la opción `--batching_parameters_file`).

#### -----------------------------------------------------------------------------------------------------------

Si espera obtener muchas consultas por segundo, querrá implementar TF Serving en varios servidores y equilibrar la carga de las consultas (consulte la Figura 19-2). Esto requerirá la implementación y gestión de muchos contenedores de servicio TF en estos servidores. Una forma de manejar eso es utilizar una herramienta como Kubernetes, que es un sistema de código abierto para simplificar la orquestación de contenedores en muchos servidores. Si no desea comprar, mantener y actualizar toda la infraestructura de hardware, querrá utilizar máquinas virtuales en una plataforma en la nube como Amazon AWS, Microsoft Azure, Google Cloud Platform, IBM Cloud, Alibaba Cloud, Oracle Cloud o alguna otra oferta de plataforma como servicio (PaaS). Gestionar todas las máquinas virtuales, manejar la orquestación de contenedores (incluso con la ayuda de Kubernetes), ocuparse de la configuración de servicio de TF, ajustar y supervisar, todo esto puede ser un trabajo a tiempo completo. Afortunadamente, algunos proveedores de servicios pueden encargarse de todo esto por usted. En este capítulo usaremos Vertex AI: es la única plataforma con TPU hoy en día; es compatible con TensorFlow 2, Scikit-Learn y XGBoost; y ofrece un buen conjunto de servicios de IA. Sin embargo, hay varios otros proveedores en este espacio que también son capaces de servir a los modelos de TensorFlow, como Amazon AWS SageMaker y Microsoft AI Platform, así que asegúrate de revisarlos también.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1902.png)

(_Figura 19-2. Ampliación de la porción de TF con equilibrio de carga_)

¡Ahora veamos cómo servir a nuestro maravilloso modelo MNIST en la nube!


### Creación de un servicio de predicción en Vertex AI

Vertex AI es una plataforma dentro de Google Cloud Platform (GCP) que ofrece una amplia gama de herramientas y servicios relacionados con la IA. Puede cargar conjuntos de datos, hacer que los humanos los etiqueten, almacenar características de uso común en una tienda de características y usarlas para entrenamiento o en producción, y entrenar modelos en muchos servidores de GPU o TPU con ajuste automático de hiperparámetros o búsqueda de arquitectura de modelos (AutoML). También puede administrar sus modelos entrenados, usarlos para hacer predicciones por lotes sobre grandes cantidades de datos, programar múltiples trabajos para sus flujos de trabajo de datos, servir sus modelos a través de REST o gRPC a escala, y experimentar con sus datos y modelos dentro de un entorno Jupyter alojado llamado theWorkbench. Incluso hay un servicio de Matching Engine que te permite comparar vectores de manera muy eficiente (es decir, los vecinos más cercanos). GCP también incluye otros servicios de IA, como API para visión por ordenador, traducción, voz a texto y más.

Antes de empezar, hay que ocuparnos de un poco de configuración:

1. Inicie sesión en su cuenta de Google y luego vaya a la consola de Google Cloud Platform (consulte la Figura 19-3). Si no tienes una cuenta de Google, tendrás que crear una.

2. Si es la primera vez que usas GCP, tendrás que leer y aceptar los términos y condiciones. A los nuevos usuarios se les ofrece una prueba gratuita, que incluye un crédito GCP por valor de 300 dólares que puede utilizar en el transcurso de 90 días (a partir de mayo de 2022). Solo necesitarás una pequeña parte de eso para pagar los servicios que utilizarás en este capítulo. Al registrarse para una prueba gratuita, aún tendrá que crear un perfil de pago e ingresar su número de tarjeta de crédito: se utiliza con fines de verificación, probablemente para evitar que las personas usen la prueba gratuita varias veces, pero no se le facturarán los primeros 300 dólares, y después de eso solo se le cobrará si opta por actualizar a una cuenta de pago.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1903.png)

(_Figura 19-3. Consola de Google Cloud Platform_)

3. Si ha utilizado GCP antes y su prueba gratuita ha caducado, entonces los servicios que utilizará en este capítulo le costarán algo de dinero. No debería ser demasiado, especialmente si recuerdas apagar los servicios cuando ya no los necesitas. Asegúrese de entender y aceptar las condiciones de precios antes de ejecutar cualquier servicio. ¡Por la presente rechazo cualquier responsabilidad si los servicios terminan costando más de lo que esperabas! Asegúrate también de que tu cuenta de facturación esté activa. Para comprobarlo, abra el menú de navegación ☰ en la parte superior izquierda y haga clic en Facturación, luego asegúrese de haber configurado un método de pago y de que la cuenta de facturación esté activa.

4. Cada recurso de GCP pertenece a un proyecto. Esto incluye todas las máquinas virtuales que puede utilizar, los archivos que almacena y los trabajos de formación que ejecuta. Cuando creas una cuenta, GCP crea automáticamente un proyecto para ti, llamado "Mi primer proyecto". Si lo desea, puede cambiar su nombre para mostrar yendo a la configuración del proyecto: en el menú de navegación ☰, seleccione "IAM y admin → Configuración", cambie el nombre para mostrar del proyecto y haga clic en GUARDAR. Tenga en cuenta que el proyecto también tiene un ID y un número únicos. Puedes elegir el ID del proyecto cuando crees un proyecto, pero no puedes cambiarlo más tarde. El número de proyecto se genera automáticamente y no se puede cambiar. Si desea crear un nuevo proyecto, haga clic en el nombre del proyecto en la parte superior de la página, luego haga clic en NUEVO PROYECTO e introduzca el nombre del proyecto. También puedes hacer clic en EDITAR para establecer el ID del proyecto. Asegúrese de que la facturación esté activa para este nuevo proyecto para que se puedan facturar las tarifas de servicio (a sus créditos gratuitos, si los hay).

#### ADVERTENCIA

Siempre configure una alarma para recordarse a sí mismo que debe apagar los servicios cuando sepa que solo los necesitará durante unas pocas horas, o de lo contrario podría dejarlos en funcionamiento durante días o meses, incurriendo en costos potencialmente significativos.

#### --------------------------------------------------------------------------------------------------------------

5. Ahora que tiene una cuenta GCP y un proyecto, y la facturación está activada, debe activar las API que necesita. En el menú de navegación ☰, seleccione "API y servicios" y asegúrese de que la API de almacenamiento en la nube esté habilitada. Si es necesario, haga clic en + HABILITAR APIS Y SERVICIOS, encuentre Almacenamiento en la nube y habilite. También habilite la API de IA de vértices.

Podrías seguir haciendo todo a través de la consola de GCP, pero recomiendo usar Python en su lugar: de esta manera puedes escribir scripts para automatizar casi cualquier cosa que quieras con GCP, y a menudo es más conveniente que hacer clic en los menús y formularios, especialmente para tareas comunes.

#### GOOGLE CLOUD CLI Y SHELL ------------------------------------------------------------------------

La interfaz de línea de comandos (CLI) de Google Cloud incluye el comando `gcloud`, que le permite controlar casi todo en GCP, y `gsutil`, que le permite interactuar con Google Cloud Storage. Esta CLI está preinstalada en Colab: todo lo que necesita hacer es autenticarse usando `google.auth.authenticate_user()` y listo. Por ejemplo, la lista de configuración de `!gcloud` mostrará la configuración.

GCP también ofrece un entorno de shell preconfigurado llamado Google Cloud Shell, que puede usar directamente en su navegador web; se ejecuta en una máquina virtual de Linux gratuita (Debian) con el SDK de Google Cloud ya preinstalado y configurado para usted, por lo que no hay necesidad de autenticarse. El Cloud Shell está disponible en cualquier lugar de GCP: simplemente haga clic en el icono Activar Cloud Shell en la parte superior derecha de la página (ver Figura 19-4).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1904.png)

(_Figura 19-4. Activación de Google Cloud Shell_)

Si prefiere instalar la CLI en su máquina, después de la instalación, debe inicializarla ejecutando `gcloud init`: siga las instrucciones para iniciar sesión en GCP y conceder acceso a sus recursos de GCP, luego seleccione el proyecto GCP predeterminado que desea utilizar (si tiene más de uno) y la región predeterminada donde desea que se ejecuten sus trabajos.

#### -------------------------------------------------------------------------------------------------------------

Lo primero que debe hacer antes de poder usar cualquier servicio de GCP es autenticarse. La solución más simple al usar Colab es ejecutar el siguiente código:

In [ ]:
from google.colab import auth

auth.authenticate_user()

El proceso de autenticación se basa en OAuth 2.0: una ventana emergente le pedirá que confirme que desea que el cuaderno Colab acceda a sus credenciales de Google. Si aceptas, debes seleccionar la misma cuenta de Google que usaste para GCP. A continuación, se le pedirá que confirme que acepta dar a Colab acceso completo a todos sus datos en Google Drive y en GCP. Si permite el acceso, solo el portátil actual tendrá acceso, y solo hasta que expire el tiempo de ejecución de Colab. Obviamente, solo debes aceptar esto si confías en el código del cuaderno.

#### ADVERTENCIA

Si no estás trabajando con los cuadernos oficiales de https://github.com/ageron/handson-ml3, entonces debes tener mucho cuidado: si el autor del cuaderno es travieso, podrían incluir código para hacer lo que quieran con tus datos.

#### --------------------------------------------------------------------------------------------------------------

#### AUTENTICACIÓN Y AUTORIZACIÓN EN GCP

En general, el uso de la autenticación OAuth 2.0 solo se recomienda cuando una aplicación debe acceder a los datos o recursos personales del usuario desde otra aplicación, en nombre del usuario. Por ejemplo, algunas aplicaciones permiten al usuario guardar datos en su Google Drive, pero para eso la aplicación primero necesita que el usuario se autentique con Google y permita el acceso a Google Drive. En general, la aplicación solo solicitará el nivel de acceso que necesita; no será un acceso ilimitado: por ejemplo, la aplicación solo solicitará acceso a Google Drive, no a Gmail o a cualquier otro servicio de Google. Además, la autorización suele expirar después de un tiempo, y siempre se puede revocar.

Cuando una aplicación necesita acceder a un servicio en GCP en su propio nombre, no en nombre del usuario, entonces generalmente debe usar una cuenta de servicio. Por ejemplo, si construye un sitio web que necesita enviar solicitudes de predicción a un punto final de Vertex AI, entonces el sitio web accederá al servicio en su propio nombre. No hay datos o recursos a los que necesite acceder en la cuenta de Google del usuario. De hecho, muchos usuarios del sitio web ni siquiera tendrán una cuenta de Google. Para este escenario, primero debe crear una cuenta de servicio. Seleccione "IAM y admin → Cuentas de servicio" en el menú de navegación ☰ de la consola GCP (o utilice el cuadro de búsqueda), luego haga clic en + CREAR CUENTA DE SERVICIO, rellene la primera página del formulario (nombre de la cuenta de servicio, ID, descripción) y haga clic en CREAR Y CONTINUAR. A continuación, debe otorgar a esta cuenta algunos derechos de acceso. Seleccione el rol de "usuario de IA de vértices": esto permitirá que la cuenta de servicio haga predicciones y utilice otros servicios de IA de vértices, pero nada más. Haga clic en CONTINUAR. Ahora puede conceder opcionalmente a algunos usuarios acceso a la cuenta de servicio: esto es útil cuando su cuenta de usuario de GCP es parte de una organización y desea autorizar a otros usuarios de la organización a implementar aplicaciones que se basarán en esta cuenta de servicio, o a administrar la cuenta de servicio en sí. A continuación, haga clic en HECHO.

Una vez que haya creado una cuenta de servicio, su aplicación debe autenticarse como esa cuenta de servicio. Hay varias formas de hacerlo. Si su aplicación está alojada en GCP, por ejemplo, si está codificando un sitio web alojado en Google Compute Engine, entonces la solución más simple y segura es adjuntar la cuenta de servicio al recurso de GCP que aloja su sitio web, como una instancia de VM o un servicio de Google App Engine. Esto se puede hacer al crear el recurso GCP, seleccionando la cuenta de servicio en la sección "Identidad y acceso a la API". Algunos recursos, como las instancias de VM, también le permiten adjuntar la cuenta de servicio después de crear la instancia de VM: debe detenerla y editar su configuración. En cualquier caso, una vez que se adjunta una cuenta de servicio a una instancia de VM, o a cualquier otro recurso de GCP que ejecute su código, las bibliotecas de clientes de GCP (que se discutirán en breve) se autenticarán automáticamente como la cuenta de servicio elegida, sin necesidad de un paso adicional.

Si su aplicación está alojada en Kubernetes, debe utilizar el servicio Workload Identity de Google para asignar la cuenta de servicio correcta a cada cuenta de servicio de Kubernetes. Si su aplicación no está alojada en GCP (por ejemplo, si solo está ejecutando el cuaderno Jupyter en su propia máquina), puede usar el servicio Workload Identity Federation (esa es la opción más segura pero más difícil) o simplemente generar una clave de acceso. para su cuenta de servicio, guárdelo en un archivo JSON y apúntelo con la variable de entorno `GOOGLE_APPLICATION_CREDENTIALS` para que su aplicación cliente pueda acceder a él. Puede administrar las claves de acceso haciendo clic en la cuenta de servicio que acaba de crear y luego abriendo la pestaña LLAVES. Asegúrese de mantener el archivo de clave en secreto: es como una contraseña para la cuenta de servicio.

Para obtener más detalles sobre cómo configurar la autenticación y la autorización para que su aplicación pueda acceder a los servicios de GCP, consulte la documentación.

#### --------------------------------------------------------------------------------------------------------------

Ahora vamos a crear un cubo de almacenamiento en la nube de Google para almacenar nuestros modelos guardados (un cubo de GCS es un contenedor para sus datos). Para esto utilizaremos la biblioteca `google-cloud-storage`, que está preinstalada en Colab. Primero creamos un objeto `Client`, que servirá como interfaz con GCS, luego lo usamos para crear el cubo:


In [ ]:
from google.cloud import storage

project_id = "my_project"  # change this to your project ID
bucket_name = "my_bucket"  # change this to a unique bucket name
location = "us-central1"

storage_client = storage.Client(project=project_id)
bucket = storage_client.create_bucket(bucket_name, location=location)

#### TIP

Si desea reutilizar un depósito existente, reemplace la última línea con `bucket = storage_client.bucket(bucket_name)`. Asegúrese de que la `location` esté configurada en la región del depósito.

#### ---------------------------------------------------------------------------------------------------------------

GCS utiliza un solo espacio de nombres mundial para los cubos, por lo que lo más probable es que nombres simples como "aprendizaje automático" no estén disponibles. Asegúrese de que el nombre del cubo se ajuste a las convenciones de nomenclatura DNS, ya que se puede utilizar en los registros DNS. Además, los nombres de los cubos son públicos, así que no pongas nada privado en el nombre. Es común usar su nombre de dominio, el nombre de su empresa o el ID de su proyecto como prefijo para garantizar la singularidad, o simplemente usar un número aleatorio como parte del nombre.

Puedes cambiar la región si quieres, pero asegúrate de elegir una que admita GPU. Además, es posible que desee considerar el hecho de que los precios varían mucho entre regiones, algunas regiones producen mucho más CO2 que otras, algunas regiones no admiten todos los servicios y el uso de un cubo de una sola región mejora el rendimiento. Consulte la lista de regiones de Google Cloud y la documentación de Vertex AI sobre ubicaciones para obtener más detalles. Si no estás seguro, podría ser mejor seguir con `"us-central1"`.

A continuación, subamos el directorio my_mnist_model al nuevo depósito. Los archivos en GCS se llaman blobs (u objetos), y bajo el capó todos se colocan en el cubo sin ninguna estructura de directorios. Los nombres de blob pueden ser cadenas Unicode arbitrarias, e incluso pueden contener barras diagonales hacia adelante (/). La consola de GCP y otras herramientas utilizan estas barras diagonales para dar la ilusión de que hay directorios. Por lo tanto, cuando subimos el directorio my_mnist_model, solo nos importan los archivos, no los directorios:

In [ ]:
def upload_directory(bucket, dirpath):
    dirpath = Path(dirpath)
    for filepath in dirpath.glob("**/*"):
        if filepath.is_file():
            blob = bucket.blob(filepath.relative_to(dirpath.parent).as_posix())
            blob.upload_from_filename(filepath)

upload_directory(bucket, "my_mnist_model")

Esta función funciona bien ahora, pero sería muy lenta si hubiera muchos archivos para subir. No es demasiado difícil acelerarlo enormemente con el subproceso múltiple (consulte el cuaderno para una implementación). Alternativamente, si tiene la CLI de Google Cloud, puede usar el siguiente comando en su lugar:

In [ ]:
!gsutil -m cp -r my_mnist_model gs://{bucket_name}/

A continuación, hablemos de Vertex AI sobre nuestro modelo MNIST. Para comunicarnos con Vertex AI, podemos usar la biblioteca `google-cloud-aiplatform` (todavía usa el antiguo nombre de AI Platform en lugar de Vertex AI). No está preinstalado en Colab, por lo que debemos instalarlo. Después de eso, podemos importar la biblioteca e inicializarla, solo para especificar algunos valores predeterminados para el ID del proyecto y la ubicación, luego podemos crear un nuevo modelo de Vertex AI: especificamos un nombre para mostrar, la ruta GCS a nuestro modelo (en en este caso la versión 0001) y la URL del contenedor Docker que queremos que Vertex AI use para ejecutar este modelo. Si visita esa URL y navega hacia arriba un nivel, encontrará otros contenedores que puede usar. Este es compatible con TensorFlow 2.8 con una GPU:

In [ ]:
from google.cloud import aiplatform

server_image = "gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-8:latest"

aiplatform.init(project=project_id, location=location)
mnist_model = aiplatform.Model.upload(
    display_name="mnist",
    artifact_uri=f"gs://{bucket_name}/my_mnist_model/0001",
    serving_container_image_uri=server_image,
)

Ahora implementemos este modelo para que podamos consultarlo a través de una API gRPC o REST para hacer predicciones. Para esto, primero necesitamos crear un punto final. Esto es a lo que se conectan las aplicaciones cliente cuando quieren acceder a un servicio. Entonces tenemos que implementar nuestro modelo en este punto final:

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name="mnist-endpoint")

endpoint.deploy(
    mnist_model,
    min_replica_count=1,
    max_replica_count=5,
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1
)

Este código puede tardar unos minutos en ejecutarse, porque Vertex AI necesita configurar una máquina virtual. En este ejemplo, utilizamos una máquina bastante básica de tipo `n1-standard-4` (consulte https://homl.info/machinetypes para conocer otros tipos). También utilizamos una GPU básica de tipo `NVIDIA_TESLA_K80` (consulte https://homl.info/accelerators para conocer otros tipos). Si seleccionó otra región que no sea `"us-central1"`, es posible que deba cambiar el tipo de máquina o el tipo de acelerador a valores admitidos en esa región (por ejemplo, no todas las regiones tienen GPU Nvidia Tesla K80).

#### NOTA

Google Cloud Platform aplica varias cuotas de GPU, tanto en todo el mundo como por región: no se pueden crear miles de nodos de GPU sin la autorización previa de Google. Para comprobar sus cuotas, abra "IAM y admin → Cuotas" en la consola de GCP. Si algunas cuotas son demasiado bajas (por ejemplo, si necesita más GPU en una región en particular), puede solicitar que se aumenten; a menudo toma alrededor de 48 horas.

#### --------------------------------------------------------------------------------------------------------------

Vertex AI generará inicialmente el número mínimo de nodos de computación (solo uno en este caso), y cada vez que el número de consultas por segundo sea demasiado alto, generará más nodos (hasta el número máximo que definió, cinco en este caso) y equilibrará la carga de las consultas entre ellos. Si la tasa de QPS baja durante un tiempo, Vertex AI detendrá los nodos de cómputo adicionales automáticamente. Por lo tanto, el costo está directamente relacionado con la carga, así como con los tipos de máquina y acelerador que seleccionó y la cantidad de datos que almacena en GCS. Este modelo de precios es ideal para usuarios ocasionales y para servicios con importantes picos de uso. También es ideal para startups: el precio sigue siendo bajo hasta que la startup realmente se inicie.

¡Enhorabuena, has implementado tu primer modelo en la nube! Ahora vamos a consultar este servicio de predicción:

In [ ]:
response = endpoint.predict(instances=X_new.tolist())

Primero tenemos que convertir las imágenes que queremos clasificar en una lista de Python, como lo hicimos antes cuando enviamos solicitudes a TF Serving utilizando la API REST. El objeto de respuesta contiene las predicciones, representadas como una lista de Python de listas de flotantes. Vamos a redondearlos a dos decimales y convertirlos en una matriz NumPy:

In [ ]:
import numpy as np
np.round(response.predictions, 2)

#array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
#       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
#       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

¡Sí! Tenemos exactamente las mismas predicciones que antes. Ahora tenemos un buen servicio de predicción que se ejecuta en la nube que podemos consultar desde cualquier lugar de forma segura, y que puede escalar automáticamente hacia arriba o hacia abajo dependiendo del número de QPS. Cuando termines de usar el punto final, no olvides eliminarlo, para evitar pagar por nada:

In [ ]:
endpoint.undeploy_all()  # undeploy all models from the endpoint
endpoint.delete()

Ahora veamos cómo ejecutar un trabajo en Vertex AI para hacer predicciones sobre un lote de datos potencialmente muy grande.


### Ejecución de trabajos de predicción por lotes en Vertex AI

Si tenemos un gran número de predicciones que hacer, entonces en lugar de llamar a nuestro servicio de predicción repetidamente, podemos pedirle a Vertex AI que ejecute un trabajo de predicción para nosotros. Esto no requiere un punto final, solo un modelo. Por ejemplo, ejecutemos un trabajo de predicción en las primeras 100 imágenes del conjunto de pruebas, utilizando nuestro modelo MNIST. Para esto, primero tenemos que preparar el lote y subirlo a GCS. Una forma de hacerlo es crear un archivo que contenga una instancia por línea, cada una con formato de valor JSON (este formato se llama JSON Lines) y luego pasar este archivo a Vertex AI. Así que vamos a crear un archivo JSON Lines en un nuevo directorio, y luego vamos a subir este directorio a GCS:

In [ ]:
batch_path = Path("my_mnist_batch")
batch_path.mkdir(exist_ok=True)
with open(batch_path / "my_mnist_batch.jsonl", "w") as jsonl_file:
    for image in X_test[:100].tolist():
        jsonl_file.write(json.dumps(image))
        jsonl_file.write("\n")

upload_directory(bucket, batch_path)

Ahora estamos listos para lanzar el trabajo de predicción, especificando el nombre del trabajo, el tipo y el número de máquinas y aceleradores a usar, la ruta de GCS al archivo JSON Lines que acabamos de crear y la ruta al directorio GCS donde Vertex AI guardará las predicciones del modelo:

In [ ]:
batch_prediction_job = mnist_model.batch_predict(
    job_display_name="my_batch_prediction_job",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=5,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=1,
    gcs_source=[f"gs://{bucket_name}/{batch_path.name}/my_mnist_batch.jsonl"],
    gcs_destination_prefix=f"gs://{bucket_name}/my_mnist_predictions/",
    sync=True  # set to False if you don't want to wait for completion
)

#### TIP

Para lotes grandes, puede dividir las entradas en varios archivos de líneas JSON y enumerarlos todos a través del argumento `gcs_source`.

#### ---------------------------------------------------------------------------------------------------------------

Esto tomará unos minutos, principalmente para generar los nodos de computación en Vertex AI. Una vez que se complete este comando, las predicciones estarán disponibles en un conjunto de archivos llamado algo así como prediction.results-00001of-00002. Estos archivos utilizan el formato JSON Lines de forma predeterminada, y cada valor es un diccionario que contiene una instancia y su predicción correspondiente (es decir, 10 probabilidades). Las instancias se enumeran en el mismo orden que las entradas. El trabajo también genera archivos de errores de predicción*, que pueden ser útiles para depurar si algo sale mal. Podemos iterar a través de todos estos archivos de salida usandobatch `batch_prediction_job.iter_outputs()`, así que revisemos todas las predicciones y almacenémoslas en una matriz `y_probas`:

In [ ]:
y_probas = []
for blob in batch_prediction_job.iter_outputs():
    if "prediction.results" in blob.name:
        for line in blob.download_as_text().splitlines():
            y_proba = json.loads(line)["prediction"]
            y_probas.append(y_proba)

Ahora veamos qué tan buenas son estas predicciones:

In [ ]:
y_pred = np.argmax(y_probas, axis=1)
accuracy = np.sum(y_pred == y_test[:100]) / 100

#0.98

¡Genial, 98 % de precisión!

El formato JSON Lines es el predeterminado, pero cuando se trata de instancias grandes, como imágenes, es demasiado detallado. Afortunadamente, el método `batch_predict()` acepta un argumento `instances_format` que le permite elegir otro formato si lo desea. El valor predeterminado es `"jsonl"`, pero puedes cambiarlo a `"csv"`, `"tf-record"`, `"tf-record-gzip"`, `"bigquery"` o `"file-list"`. Si lo configura en "lista de archivos", entonces el argumento `gcs_source` debería apuntar a un archivo de texto que contiene una ruta de archivo de entrada por línea; por ejemplo, apuntando a archivos de imagen PNG. Vertex AI leerá estos archivos como binarios, los codificará usando Base64 y pasará las cadenas de bytes resultantes al modelo. Esto significa que debe agregar una capa de preprocesamiento en su modelo para analizar las cadenas Base64, usando `tf.io.decode_base64()`. Si los archivos son imágenes, debe analizar el resultado usando una función como `tf.io.decode_image()` o `tf.io.decode_png()`, como se analiza en el Capítulo 13.

Cuando haya terminado de usar el modelo, puede eliminarlo si lo desea, ejecutando `mnist_model.delete()`). También puede eliminar los directorios que creó en su depósito de GCS, opcionalmente el propio depósito (si está vacío) y el trabajo de predicción por lotes:

In [ ]:
for prefix in ["my_mnist_model/", "my_mnist_batch/", "my_mnist_predictions/"]:
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        blob.delete()

bucket.delete()  # if the bucket is empty
batch_prediction_job.delete()

Ahora sabes cómo implementar un modelo en Vertex AI, crear un servicio de predicción y ejecutar trabajos de predicción por lotes. Pero, ¿qué pasa si quieres implementar tu modelo en una aplicación móvil en su lugar? ¿O a un dispositivo integrado, como un sistema de control de calefacción, un rastreador de actividad física o un coche autónomo?


# Implementación de un modelo en un dispositivo móvil o integrado


Los modelos de aprendizaje automático no se limitan a ejecutarse en grandes servidores centralizados con múltiples GPU: pueden ejecutarse más cerca de la fuente de datos (esto se llama computación perimetral), por ejemplo, en el dispositivo móvil del usuario o en un dispositivo integrado. Hay muchos beneficios al descentralizar los cálculos y moverlos hacia el borde: permite que el dispositivo sea inteligente incluso cuando no está conectado a Internet, reduce la latencia al no tener que enviar datos a un servidor remoto y reduce la carga en los servidores, y puede mejorar la privacidad, ya que los datos del usuario pueden permanecer en el dispositivo.

Sin embargo, el despliegue de modelos hasta el borde también tiene sus inconvenientes. Los recursos informáticos del dispositivo son generalmente pequeños en comparación con un robusto servidor multi-GPU. Un modelo grande puede no caber en el dispositivo, puede usar demasiada RAM y CPU, y puede tardar demasiado en descargarse. Como resultado, la aplicación puede no responder y el dispositivo puede calentarse y quedarse sin batería rápidamente. Para evitar todo esto, necesita hacer un modelo ligero y eficiente, sin sacrificar demasiado su precisión. La biblioteca TFLite proporciona varias herramientas⁠ para ayudarle a implementar sus modelos hasta el borde, con tres objetivos principales:

- Reduzca el tamaño del modelo para acortar el tiempo de descarga y el uso de RAM.

* Reduzca la cantidad de cálculos necesarios para cada predicción, para reducir la latencia, el uso de la batería y la calefacción.

- Adapte el modelo a las restricciones específicas del dispositivo.

Para reducir el tamaño del modelo, el convertidor de modelo de TFLite puede tomar un SavedModel y comprimirlo a un formato mucho más ligero basado en FlatBuffers. Esta es una eficiente biblioteca de serialización multiplataforma (un poco como los búferes de protocolo) creada inicialmente por Google para juegos. Está diseñado para que pueda cargar FlatBuffers directamente en la RAM sin ningún preprocesamiento: esto reduce el tiempo de carga y la huella de memoria. Una vez que el modelo se carga en un dispositivo móvil o integrado, el intérprete TFLite lo ejecutará para hacer predicciones. Así es como puedes convertir un SavedModel en un FlatBuffer y guardarlo en un archivo .tflite:

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open("my_converted_savedmodel.tflite", "wb") as f:
    f.write(tflite_model)

#### TIP

También puede guardar un modelo de Keras directamente en un FlatBuffer usando `tf.lite.TFLiteConverter.from_keras_model(model)`

#### ---------------------------------------------------------------------------------------------------------------

El convertidor también optimiza el modelo, tanto para reducirlo como para reducir su latencia. Recorta todas las operaciones que no son necesarias para hacer predicciones (como las operaciones de entrenamiento), y optimiza los cálculos siempre que sea posible; por ejemplo, 3 × a + 4 ×_ a_ + 5 × a se convertirá a 12 × a. Además, intenta fusionar las operaciones siempre que sea posible. Por ejemplo, si es posible, las capas de normalización por lotes terminan dobladas en las operaciones de adición y multiplicación de la capa anterior. Para tener una buena idea de cuánto TFLite puede optimizar un modelo, descargue uno de los modelos TFLite preentrenados, como Inception_V1_quant (haga clic en tflite&pb), descomprima el archivo, luego abra la excelente herramienta de visualización de gráficos de Netron y cargue el archivo .pb para ver el modelo original. Es un gráfico grande y complejo, ¿verdad? A continuación, ¡abre el modelo optimizado .tflite y maravíllate de su belleza!

Otra forma de reducir el tamaño del modelo, aparte de simplemente usar arquitecturas de redes neuronales más pequeñas, es mediante el uso de anchos de bits más pequeños: por ejemplo, si usa medio flotadores (16 bits) en lugar de flotadores regulares (32 bits), el tamaño del modelo se reducirá en un factor de 2, a costa de una caída de precisión (generalmente pequeña). Además, el entrenamiento será más rápido y usarás aproximadamente la mitad de la cantidad de RAM de la GPU.

¡El convertidor de TFLite puede ir más allá de eso, cuantificando los pesos del modelo a enteros de punto fijo y 8 bits! Esto conduce a una reducción de tamaño de cuatro veces en comparación con el uso de flotadores de 32 bits. El enfoque más simple se llama cuantificación posterior al entrenamiento: solo cuantifica los pesos después del entrenamiento, utilizando una técnica de cuantificación simétrica bastante básica pero eficiente. Encuentra el valor máximo de peso absoluto, m, luego mapea el rango de coma flotante -m a +m al rango de punto fijo (entero) -127 a +127. Por ejemplo, si los pesos oscilan entre -1,5 y +0,8, entonces los bytes -127, 0 y +127 corresponderán a los flotantes -1.5, 0.0 y +1.5, respectivamente (ver Figura 19-5). Tenga en cuenta que 0.0 siempre se asigna a 0 cuando se utiliza la cuantificación simétrica. También tenga en cuenta que los valores de bytes +68 a +127 no se utilizarán en este ejemplo, ya que se asignan a flotadores mayores de +0,8.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1905.png)

(_Figura 19-5. De flotantes de 32 bits a enteros de 8 bits, utilizando cuantificación simétrica_)

Para realizar esta cuantificación posterior al entrenamiento, simplemente agregue `DEFAULT` a la lista de optimizaciones del convertidor antes de llamar al método `convert()`:

In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]

Esta técnica reduce drásticamente el tamaño del modelo, lo que hace que sea mucho más rápido de descargar y utiliza menos espacio de almacenamiento. En tiempo de ejecución, los pesos cuantificados se convierten de nuevo en flotadores antes de que se utilicen. Estos flotadores recuperados no son perfectamente idénticos a los flotadores originales, pero no están demasiado lejos, por lo que la pérdida de precisión suele ser aceptable. Para evitar volver a calcular los valores flotantes todo el tiempo, lo que ralentizaría gravemente el modelo, TFLite los almacena en caché: desafortunadamente, esto significa que esta técnica no reduce el uso de RAM, y tampoco acelera el modelo. Es muy útil reducir el tamaño de la aplicación.

La forma más efectiva de reducir la latencia y el consumo de energía es también cuantificar las activaciones para que los cálculos se puedan hacer completamente con números enteros, sin necesidad de ninguna operación de coma flotante. Incluso cuando se utiliza el mismo ancho de bits (por ejemplo, enteros de 32 bits en lugar de flotadores de 32 bits), los cálculos de enteros utilizan menos ciclos de CPU, consumen menos energía y producen menos calor. Y si también reduces el ancho de bits (por ejemplo, hasta números enteros de 8 bits), puedes obtener grandes aceleraciones. Además, algunos dispositivos aceleradores de redes neuronales, como el Edge TPU de Google, solo pueden procesar números enteros, por lo que la cuantificación completa de los pesos y las activaciones es obligatoria. Esto se puede hacer después del entrenamiento; requiere un paso de calibración para encontrar el valor absoluto máximo de las activaciones, por lo que debe proporcionar una muestra representativa de datos de entrenamiento a TFLite (no tiene que ser enorme), y procesará los datos a través del modelo y medirá las estadísticas de activación requeridas para la cuantificación. Este paso suele ser rápido.

El principal problema con la cuantificación es que pierde un poco de precisión: es similar a añadir ruido a los pesos y activaciones. Si la caída de la precisión es demasiado grave, es posible que deba utilizar un entrenamiento consciente de la cuantificación. Esto significa agregar operaciones de cuantización falsas al modelo para que pueda aprender a ignorar el ruido de la cuantificación durante el entrenamiento; los pesos finales serán entonces más robustos para la cuantificación. Además, el paso de calibración se puede llevar a cabo automáticamente durante el entrenamiento, lo que simplifica todo el proceso.

He explicado los conceptos básicos de TFLite, pero llegar hasta la codificación de una aplicación móvil o integrada requeriría un libro dedicado. Afortunadamente, existen algunos: si quieres aprender más sobre la construcción de aplicaciones TensorFlow para dispositivos móviles e integrados, echa un vistazo a los libros de O'Reilly TinyML: Machine Learning with TensorFlow on Arduino y Ultra-Low Power Micro-Controllers, de Pete Warden (ex líder del equipo de TFLite) y Daniel Situnayake y AI and Machine Learning for On-Device Development, de Laurence Moroney.

Ahora, ¿qué pasa si quieres usar tu modelo en un sitio web, ejecutándose directamente en el navegador del usuario?


# Ejecutar un modelo en una página web


Ejecutar su modelo de aprendizaje automático en el lado del cliente, en el navegador del usuario, en lugar de en el lado del servidor, puede ser útil en muchos escenarios, como:

- Cuando su aplicación web se utiliza a menudo en situaciones en las que la conectividad del usuario es intermitente o lenta (por ejemplo, un sitio web para excursionistas), ejecutar el modelo directamente en el lado del cliente es la única manera de hacer que su sitio web sea confiable.

* Cuando necesitas que las respuestas del modelo sean lo más rápidas posible (por ejemplo, para un juego en línea). Eliminar la necesidad de consultar el servidor para hacer predicciones definitivamente reducirá la latencia y hará que el sitio web sea mucho más receptivo.

- Cuando su servicio web hace predicciones basadas en algunos datos privados de usuarios, y desea proteger la privacidad del usuario haciendo las predicciones en el lado del cliente para que los datos privados nunca tengan que salir de la máquina del usuario.

Para todos estos escenarios, puede utilizar la biblioteca JavaScript de TensorFlow.js (TFJS). Esta biblioteca puede cargar un modelo TFLite y hacer predicciones directamente en el navegador del usuario. Por ejemplo, el siguiente módulo de JavaScript importa la biblioteca TFJS, descarga un modelo de MobileNet preentrenado y utiliza este modelo para clasificar una imagen y registrar las predicciones. Puedes jugar con el código en https://homl.info/tfjscode, usando Glitch.com, un sitio web que te permite crear aplicaciones web en tu navegador de forma gratuita; haz clic en el botón VISTA PREVIA en la esquina inferior derecha de la página para ver el código en acción:

In [ ]:
import "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@latest";
import "https://cdn.jsdelivr.net/npm/@tensorflow-models/mobilenet@1.0.0";

const image = document.getElementById("image");

mobilenet.load().then(model => {
    model.classify(image).then(predictions => {
        for (var i = 0; i < predictions.length; i++) {
            let className = predictions[i].className
            let proba = (predictions[i].probability * 100).toFixed(1)
            console.log(className + " : " + proba + "%");
        }
    });
});

Incluso es posible convertir este sitio web en una aplicación web progresiva (PWA): este es un sitio web que respeta una serie de criterios⁠ que permiten verlo en cualquier navegador, e incluso instalarlo como una aplicación independiente en un dispositivo móvil. Por ejemplo, intente visitar https://homl.info/tfjswpa en un dispositivo móvil: la mayoría de los navegadores modernos le preguntarán si desea agregar la demostración de TFJS a su pantalla de inicio. Si aceptas, verás un nuevo icono en tu lista de solicitudes. Al hacer clic en este icono, se cargará el sitio web de demostración de TFJS dentro de su propia ventana, al igual que una aplicación móvil normal. Incluso se puede configurar una PWA para trabajar sin conexión, utilizando un trabajador de servicio: este es un módulo de JavaScript que se ejecuta en su propio hilo separado en el navegador e intercepta las solicitudes de red, lo que le permite almacenar en caché los recursos para que la PWA pueda ejecutarse más rápido, o incluso completamente fuera de línea. También puede enviar mensajes push, ejecutar tareas en segundo plano y más. Las PWA le permiten administrar una única base de código para la web y para dispositivos móviles. También hacen que sea más fácil asegurarse de que todos los usuarios ejecuten la misma versión de su aplicación. Puedes jugar con el código PWA de esta demostración de TFJS en Glitch.com en https://homl.info/wpacode.


#### TIP

Echa un vistazo a muchas más demostraciones de modelos de aprendizaje automático que se ejecutan en tu navegador en https://tensorflow.org/js/demos.

#### ---------------------------------------------------------------------------------------------------------------

¡TFJS también es compatible con el entrenamiento de un modelo directamente en su navegador web! Y en realidad es bastante rápido. Si su ordenador tiene una tarjeta GPU, entonces TFJS generalmente puede usarla, incluso si no es una tarjeta Nvidia. De hecho, TFJS usará WebGL cuando esté disponible, y dado que los navegadores web modernos generalmente admiten una amplia gama de tarjetas GPU, TFJS en realidad admite más tarjetas GPU que TensorFlow normal (que solo admite tarjetas Nvidia).

Entrenar un modelo en el navegador web de un usuario puede ser especialmente útil para garantizar que los datos de este usuario permanezcan privados. Un modelo se puede entrenar de forma centralizada, y luego ajustarlo localmente, en el navegador, en función de los datos de ese usuario. Si estás interesado en este tema, echa un vistazo al aprendizaje federado.

Una vez más, hacer justicia a este tema requeriría un libro completo. Si quieres obtener más información sobre TensorFlow.js, echa un vistazo a los libros de O'reilly Practical Deep Learning for Cloud, Mobile, and Edge, de Anirudh Koul et al., o Learning TensorFlow.js, de Gant Laborde.

Ahora que has visto cómo implementar los modelos de TensorFlow en TF Serving, o en la nube con Vertex AI, o en dispositivos móviles e integrados usando TFLite, o en un navegador web usando TFJS, hablemos de cómo usar las GPU para acelerar los cálculos.


# Uso de GPU para acelerar los cálculos


En el capítulo 11 analizamos varias técnicas que pueden acelerar considerablemente el entrenamiento: una mejor inicialización de peso, optimizadores sofisticados, etc. Pero incluso con todas estas técnicas, entrenar una gran red neuronal en una sola máquina con una sola CPU puede llevar horas, días o incluso semanas, dependiendo de la tarea. Gracias a las GPU, este tiempo de entrenamiento se puede reducir a minutos u horas. Esto no solo ahorra una enorme cantidad de tiempo, sino que también significa que puede experimentar con varios modelos mucho más fácilmente y, con frecuencia, volver a entrenar a sus modelos con datos nuevos.

En los capítulos anteriores, utilizamos tiempos de ejecución habilitados para GPU en Google Colab. Todo lo que tiene que hacer para esto es seleccionar "Cambiar tipo de tiempo de ejecución" en el menú Tiempo de ejecución y elegir el tipo de acelerador de la GPU; TensorFlow detecta automáticamente la GPU y la utiliza para acelerar los cálculos, y el código es exactamente el mismo que sin una GPU. Luego, en este capítulo, vio cómo implementar sus modelos en Vertex AI en múltiples nodos de computación habilitados para GPU: es solo una cuestión de seleccionar la imagen Docker correcta habilitada para GPU al crear el modelo Vertex AI, y seleccionar el tipo de GPU deseado al llamar a `endpoint.deploy()`. Pero, ¿y si quieres comprar tu propia GPU? ¿Y qué pasa si quieres distribuir los cálculos a través de la CPU y varios dispositivos GPU en una sola máquina (ver Figura 19-6)? Esto es lo que discutiremos ahora, luego más adelante en este capítulo discutiremos cómo distribuir los cálculos en varios servidores.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1906.png)

(_Figura 19-6. Ejecutar un gráfico TensorFlow a través de múltiples dispositivos en paralelo_)


## Conseguir tu propia GPU

Si sabes que vas a usar una GPU en gran medida y durante un largo período de tiempo, entonces comprar la tuya puede tener sentido financiero. También es posible que quieras entrenar a tus modelos localmente porque no quieres subir tus datos a la nube. O tal vez solo quieras comprar una tarjeta GPU para jugar, y también te gustaría usarla para el aprendizaje profundo.

Si decides comprar una tarjeta GPU, tómate un tiempo para tomar la decisión correcta. Tendrá que tener en cuenta la cantidad de RAM que necesitará para sus tareas (por ejemplo, normalmente al menos 10 GB para el procesamiento de imágenes o PNL), el ancho de banda (es decir, la rapidez con la que puede enviar datos dentro y fuera de la GPU), el número de núcleos, el sistema de refrigeración, etc. Tim Dettmers escribió una excelente entrada de blog para ayudarte a elegir: te animo a que la leas detenidamente. En el momento de escribir este artículo, TensorFlow solo es compatible con tarjetas Nvidia con CUDA Compute Capability 3.5+ (así como los TPU de Google, por supuesto), pero puede extender su soporte a otros fabricantes, así que asegúrese de revisar la documentación de TensorFlow para ver qué dispositivos son compatibles hoy en día.

Si opta por una tarjeta GPU Nvidia, tendrá que instalar los controladores Nvidia apropiados y varias bibliotecas Nvidia. Estos incluyen el kit de herramientas de la biblioteca de arquitectura de dispositivos unificados de cómputo (CUDA), que permite a los desarrolladores usar GPU habilitadas para CUDA para todo tipo de cálculos (no solo la aceleración de gráficos), y la biblioteca de red neuronal profunda de CUDA (cuDNN), una biblioteca acelerada por GPU de cálculos comunes de DNN, como capas de activación, normalización, convoluciones hacia adelante y hacia atrás, y agrupación (ver Capítulo 14). cuDNN es parte del SDK de aprendizaje profundo de Nvidia. Ten en cuenta que tendrás que crear una cuenta de desarrollador de Nvidia para poder descargarla. TensorFlow utiliza CUDA y cuDNN para controlar las tarjetas GPU y acelerar los cálculos (ver Figura 19-7).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1907.png)

(_Figura 19-7. TensorFlow utiliza CUDA y cuDNN para controlar las GPU y aumentar los DNN_)


Una vez que haya instalado las tarjetas GPU y todos los controladores y bibliotecas necesarios, puede usar el comando `nvidia-smi` para verificar que todo esté instalado correctamente. Este comando enumera las tarjetas GPU disponibles, así como todos los procesos que se ejecutan en cada tarjeta. En este ejemplo, es una tarjeta GPU Nvidia Tesla T4 con aproximadamente 15 GB de RAM disponible y no hay ningún proceso ejecutándose actualmente en ella:

In [ ]:
$ nvidia-smiSun Apr 10 04:52:10 2022
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|  No running processes found                                                 |
+-----------------------------------------------------------------------------+

Para comprobar que TensorFlow realmente ve su GPU, ejecute los siguientes comandos y asegúrese de que el resultado no esté vacío:

In [ ]:
physical_gpus = tf.config.list_physical_devices("GPU")
physical_gpus

#[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Gestión de la RAM de la GPU


De forma predeterminada, TensorFlow captura automáticamente casi toda la RAM en todas las GPU disponibles la primera vez que ejecutas un cálculo. Hace esto para limitar la fragmentación de la RAM de la GPU. Esto significa que si intenta iniciar un segundo programa TensorFlow (o cualquier programa que requiera la GPU), se quedará rápidamente sin RAM. Esto no sucede tan a menudo como podría pensar, ya que la mayoría de las veces tendrá un solo programa TensorFlow ejecutando en una máquina: generalmente un script de entrenamiento, un nodo de servicio TF o un cuaderno Jupyter. Si necesita ejecutar varios programas por alguna razón (por ejemplo, para entrenar dos modelos diferentes en paralelo en la misma máquina), entonces tendrá que dividir la RAM de la GPU entre estos procesos de manera más unisa.

Si tiene varias tarjetas GPU en su máquina, una solución sencilla es asignar cada una de ellas a un único proceso. Para hacer esto, puede configurar la variable de entorno `CUDA_VISIBLE_DEVICES` para que cada proceso solo vea las tarjetas GPU apropiadas. También configure la variable de entorno `CUDA_DEVICE_ORDER` en `PCI_BUS_ID` para garantizar que cada ID siempre haga referencia a la misma tarjeta GPU. Por ejemplo, si tiene cuatro tarjetas GPU, puede iniciar dos programas, asignando dos GPU a cada uno de ellos, ejecutando comandos como los siguientes en dos ventanas de terminal separadas:

In [ ]:
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
# and in another terminal:
$ CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_2.py

Entonces, el programa 1 solo verá las tarjetas GPU 0 y 1, denominadas `"/gpu:0"` y `"/gpu:1"`, respectivamente, en TensorFlow, y el programa 2 solo verá las tarjetas GPU 2 y 3, denominadas `"/gpu:1"` y `"/gpu:0"`, respectivamente (tenga en cuenta el orden). Todo funcionará bien (ver Figura 19-8). Por supuesto, también puedes definir estas variables de entorno en Python configurando `os.environ["CUDA_DEVICE_ORDER"]` y `os.environ["CUDA_VISI⁠BLE_DEVICES"]`, siempre que lo hagas antes de usar TensorFlow.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1908.png)

(_Figura 19-8. Cada programa tiene dos GPU_)

Otra opción es decirle a TensorFlow que tome solo una cantidad específica de RAM de la GPU. Esto debe hacerse inmediatamente después de importar TensorFlow. Por ejemplo, para que TensorFlow tome solo 2 GiB de RAM en cada GPU, debe crear un dispositivo GPU lógico (a veces llamado dispositivo GPU virtual) para cada dispositivo GPU físico y establecer su límite de memoria en 2 GiB (es decir, 2., 2.48 MiB):

In [ ]:
for gpu in physical_gpus:
    tf.config.set_logical_device_configuration(
        gpu,
        [tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
    )

Supongamos que tiene cuatro GPU, cada una con al menos 4 GiB de RAM: en este caso, dos programas como este pueden ejecutarse en paralelo, cada uno usando las cuatro tarjetas GPU (consulte la Figura 19-9). Si ejecuta el comando `nvidia-smi` mientras ambos programas se están ejecutando, debería ver que cada proceso tiene 2 GiB de RAM en cada tarjeta.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1909.png)

(_Figura 19-9. Cada programa tiene las cuatro GPU, pero con solo 2 GiB de RAM en cada GPU_)

Otra opción es decirle a TensorFlow que tome memoria solo cuando la necesite. Una vez más, esto debe hacerse inmediatamente después de importar TensorFlow:

In [ ]:
for gpu in physical_gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Otra forma de hacer esto es establecer la variable de entorno `TF_FORCE_GPU_ALLOW_GROWTH` en `true`. Con esta opción, TensorFlow nunca liberará memoria una vez que la haya capturado (nuevamente, para evitar la fragmentación de la memoria), excepto, por supuesto, cuando finalice el programa. Puede ser más difícil garantizar un comportamiento determinista usando esta opción (por ejemplo, un programa puede fallar porque el uso de memoria de otro programa se disparó), por lo que en producción probablemente querrás seguir con una de las opciones anteriores. Sin embargo, hay algunos casos en los que es muy útil: por ejemplo, cuando usas una máquina para ejecutar múltiples notebooks Jupyter, varios de los cuales usan TensorFlow. La variable de entorno `TF_FORCE_GPU_ALLOW_GROWTH` está configurada en `true` en los tiempos de ejecución de Colab.

Por último, en algunos casos es posible que desee dividir una GPU en dos o más dispositivos lógicos. Por ejemplo, esto es útil si solo tienes una GPU física, como en un tiempo de ejecución de Colab, pero quieres probar un algoritmo multi-GPU. El siguiente código divide la GPU #0 en dos dispositivos lógicos, con 2 GiB de RAM cada uno (de nuevo, esto debe hacerse inmediatamente después de importar TensorFlow):

In [ ]:
tf.config.set_logical_device_configuration(
    physical_gpus[0],
    [tf.config.LogicalDeviceConfiguration(memory_limit=2048),
     tf.config.LogicalDeviceConfiguration(memory_limit=2048)]
)

Estos dos dispositivos lógicos se llaman `"/gpu:0"` y `"/gpu:1"`, y puedes usarlos como si fueran dos GPU normales. Puede enumerar todos los dispositivos lógicos de esta manera:

In [ ]:
logical_gpus = tf.config.list_logical_devices("GPU")
logical_gpus

#[LogicalDevice(name='/device:GPU:0', device_type='GPU'),
# LogicalDevice(name='/device:GPU:1', device_type='GPU')]

Ahora veamos cómo TensorFlow decide qué dispositivos debe usar para colocar variables y ejecutar operaciones.


## Colocación de operaciones y variables en dispositivos

Keras y tf.data generalmente hacen un buen trabajo al colocar operaciones y variables donde pertenecen, pero también puede colocar operaciones y variables manualmente en cada dispositivo, si desea más control:

- Por lo general, desea colocar las operaciones de preprocesamiento de datos en la CPU y colocar las operaciones de la red neuronal en las GPU.

* Las GPU suelen tener un ancho de banda de comunicación bastante limitado, por lo que es importante evitar transferencias de datos innecesarias dentro y fuera de las GPU.

- Agregar más RAM de la CPU a una máquina es simple y bastante barato, por lo que generalmente hay mucho, mientras que la RAM de la GPU está integrada en la GPU: es un recurso costoso y, por lo tanto, limitado, por lo que si no se necesita una variable en los próximos pasos de entrenamiento, probablemente debería colocarse en la CPU (por ejemplo, los conjuntos de datos generalmente pertenecen a la CPU).

De forma predeterminada, todas las variables y todas las operaciones se colocarán en la primera GPU (la denominada `"/gpu:0"`), excepto las variables y operaciones que no tienen un núcleo de GPU:, estas se colocan en la CPU ( siempre llamado `"/cpu:0"`). El atributo de dispositivo de un tensor o variable le indica en qué `device` se colocó:⁠

In [ ]:
a = tf.Variable([1., 2., 3.])  # float32 variable goes to the GPU
a.device
#'/job:localhost/replica:0/task:0/device:GPU:0'

b = tf.Variable([1, 2, 3])  # int32 variable goes to the CPU
b.device
#'/job:localhost/replica:0/task:0/device:CPU:0'

Puede ignorar con seguridad el prefijo `/job:localhost/replica:0/task:0` por ahora; Analizaremos trabajos, réplicas y tareas más adelante en este capítulo. Como puede ver, la primera variable se colocó en la GPU n.º 0, que es el dispositivo predeterminado. Sin embargo, la segunda variable se colocó en la CPU: esto se debe a que no hay núcleos de GPU para variables enteras o para operaciones que involucran tensores enteros, por lo que TensorFlow recurrió a la CPU.

Si desea colocar una operación en un dispositivo diferente al predeterminado, utilice un contexto `tf.device()`:

In [ ]:
with tf.device("/cpu:0"):
    c = tf.Variable([1., 2., 3.])

c.device
#'/job:localhost/replica:0/task:0/device:CPU:0'

#### NOTA

La CPU siempre se trata como un único dispositivo (`"/cpu:0"`), incluso si su máquina tiene varios núcleos de CPU. Cualquier operación realizada en la CPU puede ejecutarse en paralelo en varios núcleos si tiene un núcleo multiproceso.

#### ---------------------------------------------------------------------------------------------------------------


Si intenta explícitamente colocar una operación o variable en un dispositivo que no existe o para el cual no hay un kernel, TensorFlow recurrirá silenciosamente al dispositivo que habría elegido de forma predeterminada. Esto es útil cuando desea poder ejecutar el mismo código en diferentes máquinas que no tienen la misma cantidad de GPU. Sin embargo, puede ejecutar `tf.config.set_soft_device_placement(False)` si prefiere obtener una excepción.

Ahora, ¿cómo ejecuta exactamente TensorFlow las operaciones en múltiples dispositivos?


## Ejecución Paralela A Través De Múltiples Dispositivos

Como vimos en el capítulo 12, uno de los beneficios de usar las funciones de TF es el paralelismo. Echemos un vistazo más de cerca a esto. Cuando TensorFlow ejecuta una función TF, comienza analizando su gráfico para encontrar la lista de operaciones que deben evaluarse, y cuenta cuántas dependencias tiene cada una de ellas. Luego, TensorFlow agrega cada operación con cero dependencias (es decir, cada operación de origen) a la cola de evaluación del dispositivo de esta operación (ver Figura 19-10). Una vez que se ha evaluado una operación, se disminuye el contador de dependencias de cada operación que depende de ella. Una vez que el contador de dependencias de una operación alcanza cero, se envía a la cola de evaluación de su dispositivo. Y una vez que se han calculado todas las salidas, se devuelven.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1910.png)

(_Figura 19-10. Ejecución paralelizada de un gráfico de TensorFlow_)

Las operaciones en la cola de evaluación de la CPU se envían a un grupo de subprocesos llamado grupo de subprocesos interoperatorios. Si la CPU tiene varios núcleos, estas operaciones se evaluarán efectivamente en paralelo. Algunas operaciones tienen núcleos de CPU multihilo: estos núcleos dividen sus tareas en múltiples suboperaciones, que se colocan en otra cola de evaluación y se envían a un segundo grupo de subprocesos llamado grupo de subprocesos intra-op (compartido por todos los núcleos de CPU multihilo). En resumen, se pueden evaluar múltiples operaciones y suboperaciones en paralelo en diferentes núcleos de CPU.

Para la GPU, las cosas son un poco más simples. Las operaciones en la cola de evaluación de una GPU se evalúan secuencialmente. Sin embargo, la mayoría de las operaciones tienen núcleos de GPU multihilo, normalmente implementados por bibliotecas de las que depende TensorFlow, como CUDA y cuDNN. Estas implementaciones tienen sus propios grupos de subprocesos, y normalmente explotan tantos subprocesos de GPU como pueden (que es la razón por la que no hay necesidad de un grupo de subprocesos interoperativos en las GPU: cada operación ya inunda la mayoría de los subprocesos de GPU).

Por ejemplo, en la Figura 19-10, las operaciones A, B y C son operaciones de origen, por lo que se pueden evaluar de inmediato. Las operaciones A y B se colocan en la CPU, por lo que se envían a la cola de evaluación de la CPU, luego se envían al grupo de subprocesos entre operaciones y se evalúan inmediatamente en paralelo. Resulta que la operación A tiene un núcleo multihilo; sus cálculos se dividen en tres partes, que son ejecutadas en paralelo por el grupo de subprocesos intraoperatorios. La operación C va a la cola de evaluación de la GPU #0, y en este ejemplo su núcleo de GPU utiliza cuDNN, que administra su propio grupo de subprocesos intraoperatorio y ejecuta la operación a través de muchos subprocesos de GPU en paralelo. Supongamos que C termina primero. Los contadores de dependencia de D y E se reducen y llegan a 0, por lo que ambas operaciones se envían a la cola de evaluación de la GPU #0, y se ejecutan secuencialmente. Tenga en cuenta que C solo se evalúa una vez, a pesar de que tanto D como E dependen de ello. Supongamos que B termina a continuación. Entonces el contador de dependencia de F se reduce de 4 a 3, y como eso no es 0, aún no se ejecuta. Una vez que A, D y E han terminado, el contador de dependencias de F llega a 0, y se empuja a la cola de evaluación de la CPU y se evalúa. Finalmente, TensorFlow devuelve las salidas solicitadas.

Un poco de magia adicional que realiza TensorFlow es cuando la función TF modifica un recurso con estado, como una variable: garantiza que el orden de ejecución coincida con el orden en el código, incluso si no existe una dependencia explícita entre las declaraciones. Por ejemplo, si su función TF contiene `v.assign_add(1)` seguido de `v.assign(v * 2)`, TensorFlow se asegurará de que estas operaciones se ejecuten en ese orden.

#### TIP

Puede controlar el número de hilos en el grupo de hilos interoperatorios llamando a `tf.config.threading.set_inter_op_parallelism_threads()`. Para establecer el número de subprocesos intraoperatorios, usa `tf.config.threading.set_intra_op_parallelism_threads()`. Esto es útil si no quieres que TensorFlow utilice todos los núcleos de la CPU o si quieres que sea de un solo hilo.

#### ---------------------------------------------------------------------------------------------------------------

Con eso, tienes todo lo que necesitas para ejecutar cualquier operación en cualquier dispositivo, ¡y explotar la potencia de tus GPU! Estas son algunas de las cosas que podrías hacer:

- Podrías entrenar varios modelos en paralelo, cada uno en su propia GPU: simplemente escribe un script de entrenamiento para cada modelo y ejecútalos en paralelo, configurando `CUDA_DEVICE_ORDER` y `CUDA_VISIBLE_DEVICES` para que cada script solo vea un único dispositivo GPU. Esto es excelente para el ajuste de hiperparámetros, ya que puede entrenar en paralelo varios modelos con diferentes hiperparámetros. Si tiene una sola máquina con dos GPU y se necesita una hora para entrenar un modelo en una GPU, entonces entrenar dos modelos en paralelo, cada uno en su propia GPU dedicada, tomará solo una hora. ¡Simple!

* Puede entrenar un modelo en una sola GPU y realizar todo el preprocesamiento en paralelo en la CPU, utilizando el método `prefetch()` del conjunto de datos⁠ para preparar los siguientes lotes con anticipación para que estén listos cuando la GPU los necesite (consulte el Capítulo 13).

- Si su modelo toma dos imágenes como entrada y las procesa usando dos CNN antes de unirse a sus salidas, entonces probablemente se ejecutará mucho más rápido si coloca cada CNN en una GPU diferente.

* Puedes crear un conjunto eficiente: solo tienes que colocar un modelo entrenado diferente en cada GPU para que puedas obtener todas las predicciones mucho más rápido para producir la predicción final del conjunto.

Pero, ¿y si quieres acelerar el entrenamiento usando varias GPU?


# Modelos De Entrenamiento En Múltiples Dispositivos

Hay dos enfoques principales para entrenar un solo modelo a través de múltiples dispositivos: el paralelismo del modelo, donde el modelo se divide entre los dispositivos, y el paralelismo de datos, donde el modelo se replica en cada dispositivo, y cada réplica se entrena en un subconjunto diferente de los datos. Echemos un vistazo a estas dos opciones.

## Paralelismo modelo

Hasta ahora hemos entrenado cada red neuronal en un solo dispositivo. ¿Qué pasa si queremos entrenar una sola red neuronal a través de varios dispositivos? Esto requiere cortar el modelo en trozos separados y ejecutar cada trozo en un dispositivo diferente. Desafortunadamente, tal paralelismo de modelos resulta ser bastante complicado, y su efectividad realmente depende de la arquitectura de su red neuronal. Para las redes totalmente conectadas, generalmente no hay mucho que ganar con este enfoque (ver Figura 19-11). Intuitivamente, puede parecer que una forma fácil de dividir el modelo es colocar cada capa en un dispositivo diferente, pero esto no funciona porque cada capa necesita esperar a la salida de la capa anterior antes de poder hacer algo. Así que tal vez puedas cortarlo verticalmente, por ejemplo, con la mitad izquierda de cada capa en un dispositivo y la parte derecha en otro dispositivo. Esto es un poco mejor, ya que ambas mitades de cada capa pueden funcionar en paralelo, pero el problema es que cada mitad de la siguiente capa requiere la salida de ambas mitades, por lo que habrá mucha comunicación entre dispositivos (representada por las flechas diseadas). Es probable que esto cancele por completo el beneficio del cálculo paralelo, ya que la comunicación entre dispositivos es lenta (y aún más cuando los dispositivos están ubicados en diferentes máquinas).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1911.png)

(_Figura 19-11. Dividir una red neuronal totalmente conectada_)

Algunas arquitecturas de redes neuronales, como las redes neuronales convolucionales (ver Capítulo 14), contienen capas que solo están parcialmente conectadas a las capas inferiores, por lo que es mucho más fácil distribuir trozos entre dispositivos de una manera eficiente (Figura 19-12).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1912.png)

(_Figura 19-12. Dividir una red neuronal parcialmente conectada_)

Las redes neuronales recurrentes profundas (consulte el Capítulo 15) se pueden dividir de manera un poco más eficiente en múltiples GPU. Si divide la red horizontalmente colocando cada capa en un dispositivo diferente y alimenta la red con una secuencia de entrada para procesar, entonces en el primer paso solo estará activo un dispositivo (trabajando en el primer valor de la secuencia), en el segundo El paso dos estará activo (la segunda capa manejará la salida de la primera capa para el primer valor, mientras que la primera capa manejará el segundo valor), y cuando la señal se propague a la capa de salida, todos los dispositivos estar activos simultáneamente (Figura 19-13). Todavía hay mucha comunicación entre dispositivos, pero dado que cada celda puede ser bastante compleja, el beneficio de ejecutar varias celdas en paralelo puede (en teoría) superar la penalización de la comunicación. Sin embargo, en la práctica, una pila normal de capas `LSTM` que se ejecutan en una sola GPU en realidad se ejecuta mucho más rápido.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1913.png)

(_Figura 19-13. Dividir una red neuronal recurrente profunda_)

En resumen, el paralelismo de modelos puede acelerar el funcionamiento o el entrenamiento de algunos tipos de redes neuronales, pero no todas, y requiere un cuidado y ajuste especiales, como asegurarse de que los dispositivos que necesitan comunicarse más se ejecuten en la misma máquina. A continuación, buscaremos una opción mucho más simple y generalmente más eficiente: el paralelismo de datos.


## Paralelismo de datos


Otra forma de paralelizar el entrenamiento de una red neuronal es replicarlo en cada dispositivo y ejecutar cada paso de entrenamiento simultáneamente en todas las réplicas, utilizando un mini lote diferente para cada una. Los gradientes calculados por cada réplica se promedian, y el resultado se utiliza para actualizar los parámetros del modelo. Esto se llama paralelismo de datos, o a veces un solo programa, múltiples datos (SPMD). Hay muchas variantes de esta idea, así que echemos un vistazo a las más importantes.

### Paralelismo de datos utilizando la estrategia reflejada

Podría decirse que el enfoque más simple es reflejar completamente todos los parámetros del modelo en todas las GPU y aplicar siempre las mismas actualizaciones de parámetros en cada GPU. De esta manera, todas las réplicas siempre permanecen perfectamente idénticas. Esto se llama la estrategia reflejada, y resulta ser bastante eficiente, especialmente cuando se utiliza una sola máquina (ver Figura 19-14).

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1914.png)

(_Figura 19-14. Paralelismo de datos utilizando la estrategia reflejada_)

La parte difícil al usar este enfoque es calcular de manera eficiente la media de todos los gradientes de todas las GPU y distribuir el resultado entre todas las GPU. Esto se puede hacer utilizando un algoritmo AllReduce, una clase de algoritmos en los que varios nodos colaboran para realizar de manera eficiente una operación de reducción (como calcular la media, la suma y el máximo), al tiempo que se garantiza que todos los nodos obtengan el mismo resultado final. Afortunadamente, hay implementaciones listas para usar de tales algoritmos, como verás.

### Paralelismo de datos con parámetros centralizados

Otro enfoque es almacenar los parámetros del modelo fuera de los dispositivos de la GPU que realizan los cálculos (llamados trabajadores); por ejemplo, en la CPU (ver Figura 19-15). En una configuración distribuida, puede colocar todos los parámetros en uno o más servidores solo para la CPU llamados servidores de parámetros, cuya única función es alojar y actualizar los parámetros.


![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1915.png)

(_Figura 19-15. Paralelismo de datos con parámetros centralizados_)

Mientras que la estrategia reflejada impone actualizaciones de peso sincrónicas en todas las GPU, este enfoque centralizado permite actualizaciones sincrónicas o asíncronas. Echemos un vistazo a los pros y los contras de ambas opciones.

#### Actualizaciones sincrónicas

Con las actualizaciones sincrónicas, el agregador espera hasta que todos los gradientes estén disponibles antes de calcular los gradientes promedio y pasarlos al optimizador, que actualizará los parámetros del modelo. Una vez que una réplica ha terminado de calcular sus gradientes, debe esperar a que se actualicen los parámetros antes de poder proceder al siguiente minilote. La desventaja es que algunos dispositivos pueden ser más lentos que otros, por lo que los dispositivos rápidos tendrán que esperar a los lentos en cada paso, lo que hará que todo el proceso sea tan lento como el dispositivo más lento. Además, los parámetros se copiarán en todos los dispositivos casi al mismo tiempo (inmediatamente después de que se apliquen los gradientes), lo que puede saturar el ancho de banda de los servidores de parámetros.

#### PROPINA

Para reducir el tiempo de espera en cada paso, podría ignorar los gradientes de las pocas réplicas más lentas (típicamente ~10%). Por ejemplo, podrías ejecutar 20 réplicas, pero solo agregar los gradientes de las 18 réplicas más rápidas en cada paso, y simplemente ignorar los gradientes de los últimos 2. Tan pronto como se actualicen los parámetros, las primeras 18 réplicas pueden comenzar a funcionar de nuevo de inmediato, sin tener que esperar a las 2 réplicas más lentas. Esta configuración se describe generalmente como si tuviera 18 réplicas más 2 réplicas de repuesto.

#### --------------------------------------------------------------------------------------------------------------


### Actualizaciones asíncronas

Con las actualizaciones asíncronas, cada vez que una réplica ha terminado de calcular los gradientes, los gradientes se utilizan inmediatamente para actualizar los parámetros del modelo. No hay agregación (se elimina el paso "medio" de la Figura 19-15) y no hay sincronización. Las réplicas funcionan independientemente de las otras réplicas. Dado que no hay que esperar a las otras réplicas, este enfoque realiza más pasos de entrenamiento por minuto. Además, aunque los parámetros aún deben copiarse en cada dispositivo en cada paso, esto sucede en diferentes momentos para cada réplica, por lo que se reduce el riesgo de saturación de ancho de banda.

El paralelismo de datos con actualizaciones asíncronas es una opción atractiva debido a su simplicidad, la ausencia de retraso de sincronización y su mejor uso del ancho de banda. Sin embargo, aunque funciona razonablemente bien en la práctica, ¡es casi sorprendente que funcione en absoluto! De hecho, para cuando una réplica haya terminado de calcular los gradientes basados en algunos valores de parámetros, estos parámetros habrán sido actualizados varias veces por otras réplicas (en promedio N - 1 veces, si hay N réplicas), y no hay garantía de que los gradientes calculados todavía estén apuntando en la dirección correcta (ver Figura 19-16). Cuando los gradientes están gravemente desactualizados, se llaman gradientes ranuos: pueden ralentizar la convergencia, introduciendo efectos de ruido y oscilación (la curva de aprendizaje puede contener oscilaciones temporales), o incluso pueden hacer que el algoritmo de entrenamiento diverga.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1916.png)

(_Figura 19-16. Gradientes desusos cuando se utilizan actualizaciones asíncronas_)

Hay algunas formas de reducir el efecto de los gradientes rantios:

- Reducir la tasa de aprendizaje.

* Suelte los gradientes ranios o reduzcanlos.

- Ajuste el tamaño del mini lote.

* Comienza las primeras épocas usando una sola réplica (esto se llama fase de calentamiento). Los gradientes resos tienden a ser más dañinos al comienzo del entrenamiento, cuando los gradientes suelen ser grandes y los parámetros aún no se han asentado en un valle de la función de costo, por lo que diferentes réplicas pueden empujar los parámetros en direcciones bastante diferentes.

Un documento publicado por el equipo de Google Brain en 2016⁠ comparó varios enfoques y encontró que el uso de actualizaciones sincrónicas con algunas réplicas de repuesto era más eficiente que el uso de actualizaciones asíncronas, no solo convergiendo más rápido, sino que también produciendo un mejor modelo. Sin embargo, esta sigue siendo un área activa de investigación, por lo que no debe descartar actualizaciones asíncronas por el momento.


### Saturación de ancho de banda


Ya sea que utilice actualizaciones sincrónicas o asíncronas, el paralelismo de datos con parámetros centralizados todavía requiere la comunicación de los parámetros del modelo desde los servidores de parámetros a cada réplica al comienzo de cada paso de entrenamiento, y los gradientes en la otra dirección al final de cada paso de entrenamiento. Del mismo modo, cuando se utiliza la estrategia reflejada, los gradientes producidos por cada GPU tendrán que ser compartidos con cada otra GPU. Desafortunadamente, a menudo llega un punto en el que agregar una GPU adicional no mejorará el rendimiento en absoluto porque el tiempo dedicado a mover los datos dentro y fuera de la RAM de la GPU (y a través de la red en una configuración distribuida) superará la aceleración obtenida al dividir la carga de cálculo. En ese momento, añadir más GPU solo empeorará la saturación del ancho de banda y, de hecho, ralentizará el entrenamiento.

La saturación es más grave para los modelos grandes y densos, ya que tienen muchos parámetros y gradientes para transferir. Es menos grave para los modelos pequeños (pero la ganancia de paralelización es limitada) y para los modelos grandes y escasos, donde los gradientes suelen ser en su mayoría ceros y, por lo tanto, se pueden comunicar de manera eficiente. Jeff Dean, iniciador y líder del proyecto Google Brain, informó de aceleraciones típicas de 25-40 veces al distribuir cálculos a través de 50 GPU para modelos densos, y una aceleración de 300 veces para modelos más escasos entrenados en 500 GPU. Como puedes ver, los modelos dispersos realmente escalan mejor. Aquí hay algunos ejemplos concretos:

- Traducción automática neuronal: 6 veces la aceleración en 8 GPU

* Inception/ImageNet: 32 veces la aceleración en 50 GPU

- RankBrain: 300× de aceleración en 500 GPU

Hay muchas investigaciones en curso para aliviar el problema de la saturación del ancho de banda, con el objetivo de permitir que el entrenamiento se escale linealmente con el número de GPU disponibles. Por ejemplo, un documento de 2018⁠ de un equipo de investigadores de la Universidad Carnegie Mellon, la Universidad de Stanford y Microsoft Research propuso un sistema llamado PipeDream que logró reducir las comunicaciones de red en más de un 90 %, lo que permitió entrenar modelos grandes en muchas máquinas. Lo lograron utilizando una nueva técnica llamada paralelismo de tuberías, que combina el paralelismo de modelos y el paralelismo de datos: el modelo se corta en partes consecutivas, llamadas etapas, cada una de las cuales se entrena en una máquina diferente. Esto da como resultado una tubería asíncrona en la que todas las máquinas funcionan en paralelo con muy poco tiempo de inactividad. Durante el entrenamiento, cada etapa alterna una ronda de propagación hacia adelante y una ronda de contrapropagación (ver Figura 19-17): extrae un mini lote de su cola de entrada, lo procesa y envía las salidas a la cola de entrada de la siguiente etapa, luego extrae un mini lote de gradientes de su cola de gradientes, retrocede estos gradientes y actualiza sus propios parámetros de modelo, y empuja los gradientes de propagación posterior a la cola de gradiente de la etapa anterior. Luego repite todo el proceso una y otra vez. Cada etapa también puede utilizar el paralelismo de datos regular (por ejemplo, utilizando la estrategia reflejada), independientemente de las otras etapas.

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1917.png)

(_Figura 19-17. Paralelismo de la tubería de PipeDream_)

Sin embargo, como se presenta aquí, PipeDream no funcionaría tan bien. Para entender por qué, considere el mini lote #5 en la Figura 19-17: cuando pasó por la etapa 1 durante el pase hacia adelante, los gradientes del mini lote #4 aún no se habían propagado hacia atrás a través de esa etapa, pero para el momento en que los gradientes del #5 fluyen de nuevo a la etapa 1, los gradientes del #4 se habrán utilizado para actualizar los parámetros del modelo, por lo que los gradientes del #5 estarán un poco rantilos. Como hemos visto, esto puede degradar la velocidad y la precisión del entrenamiento, e incluso hacer que diverga: cuantos más etapas haya, peor será este problema. Sin embargo, los autores del documento propusieron métodos para mitigar este problema: por ejemplo, cada etapa guarda pesos durante la propagación hacia adelante y los restaura durante la retropropagación, para garantizar que se utilicen los mismos pesos tanto para el paso hacia adelante como para el paso hacia atrás. Esto se llama estinción de peso. Gracias a esto, PipeDream demuestra una impresionante capacidad de escalado, mucho más allá del simple paralelismo de datos.

El último avance en este campo de investigación fue publicado en un artículo de 2022⁠ por investigadores de Google: desarrollaron un sistema llamado Pathways que utiliza el paralelismo de modelos automatizado, la programación asíncrona de pandillas y otras técnicas para alcanzar cerca del 100 % de la utilización de hardware a través de miles de TPU. La programación significa organizar cuándo y dónde debe ejecutarse cada tarea, y la programación de pandillas significa ejecutar tareas relacionadas al mismo tiempo en paralelo y cerca unas de otras para reducir el tiempo que las tareas tienen que esperar a los resultados de los demás. Como vimos en el capítulo 16, este sistema se utilizó para entrenar un modelo de lenguaje masivo en más de 6.000 TPU, con una utilización de casi el 100 % de hardware: esa es una hazaña de ingeniería alucinante.

En el momento de escribir este artículo, Pathways aún no es público, pero es probable que en un futuro próximo puedas entrenar enormes modelos en Vertex AI usando Pathways o un sistema similar. Mientras tanto, para reducir el problema de saturación, probablemente querrás usar algunas GPU potentes en lugar de muchas GPU débiles, y si necesitas entrenar un modelo en varios servidores, deberías agrupar tus GPU en unos pocos servidores muy bien interconectados. También puedes intentar bajar la precisión del flotador de 32 bits (`tf.float32`) a 16 bits (`tf.bfloat16`. Esto reducirá a la mitad la cantidad de datos a transferir, a menudo sin mucho impacto en la tasa de convergencia o en el rendimiento del modelo. Por último, si está utilizando parámetros centralizados, puede dividir (dividir) los parámetros en varios servidores de parámetros: agregar más servidores de parámetros reducirá la carga de la red en cada servidor y limitará el riesgo de saturación de ancho de banda.

Vale, ahora que hemos pasado por toda la teoría, ¡entrenemos un modelo a través de varias GPU!


## Formación a escala utilizando la API de estrategias de distribución

Afortunadamente, TensorFlow viene con una API muy buena que se encarga de toda la complejidad de distribuir su modelo en múltiples dispositivos y máquinas: la API de estrategias de distribución. Para entrenar un modelo de Keras en todas las GPU disponibles (en una sola máquina, por ahora) usando el paralelismo de datos con la estrategia reflejada, simplemente cree un objeto `MirroredStrategy`, llame a su método `scope()` para obtener un contexto de distribución y ajuste la creación y compilación. de su modelo dentro de ese contexto. Luego llame al método `fit()` del modelo normalmente:

In [ ]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = tf.keras.Sequential([...])  # create a Keras model normally
    model.compile([...])  # compile the model normally

batch_size = 100  # preferably divisible by the number of replicas
model.fit(X_train, y_train, epochs=10,
          validation_data=(X_valid, y_valid), batch_size=batch_size)

En el fondo, Keras tiene en cuenta la distribución, por lo que en este contexto de `MirroredStrategy` sabe que debe replicar todas las variables y operaciones en todos los dispositivos GPU disponibles. Si nos fijamos en los pesos del modelo, son del tipo `MirroredVariable`:

In [ ]:
type(model.weights[0])

#tensorflow.python.distribute.values.MirroredVariable

Tenga en cuenta que el método `fit()` dividirá automáticamente cada lote de entrenamiento entre todas las réplicas, por lo que es preferible asegurarse de que el tamaño del lote sea divisible por la cantidad de réplicas (es decir, la cantidad de GPU disponibles) para que todas las réplicas obtengan lotes de el mismo tamaño. ¡Y eso es todo! La capacitación generalmente será significativamente más rápida que usar un solo dispositivo y el cambio de código fue realmente mínimo.

Una vez que haya terminado de entrenar su modelo, puede usarlo para hacer predicciones de manera eficiente: llame al método `predict()` y automáticamente dividirá el lote entre todas las réplicas, haciendo predicciones en paralelo. Nuevamente, el tamaño del lote debe ser divisible por el número de réplicas. Si llama al método `save()` del modelo, se guardará como un modelo normal, no como un modelo reflejado con múltiples réplicas. Entonces, cuando lo cargues, se ejecutará como un modelo normal, en un solo dispositivo: de forma predeterminada en la GPU n.° 0 o en la CPU si no hay GPU. Si desea cargar un modelo y ejecutarlo en todos los dispositivos disponibles, debe llamar a `tf.keras.models.load_model()` dentro de un contexto de distribución:

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model("my_mirrored_model")

Si solo desea utilizar un subconjunto de todos los dispositivos GPU disponibles, puede pasar la lista al constructor de `MirroredStrategy`:

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

De forma predeterminada, la clase MirroredStrategy utiliza la Biblioteca de comunicaciones colectivas de NVIDIA (NCCL) para la operación media AllReduce, pero puede cambiarla estableciendo el argumento `cross_device_ops` en una instancia de la clase `tf.distribute.HierarchicalCopyAllReduce`, o una instancia de la clase `tf.distribute.ReductionToOneDevice`. La opción NCCL predeterminada se basa en la clase `tf.distribute.NcclAllReduce`, que suele ser más rápida, pero depende de la cantidad y los tipos de GPU, por lo que es posible que desees probar las alternativas.⁠

Si desea intentar utilizar el paralelismo de datos con parámetros centralizados, reemplace `MirroredStrategy` con `CentralStorageStrategy`:

In [ ]:
strategy = tf.distribute.experimental.CentralStorageStrategy()

Opcionalmente, puede configurar el argumento `compute_devices` para especificar la lista de dispositivos que desea usar como trabajadores (de forma predeterminada, usará todas las GPU disponibles) y, opcionalmente, puede configurar el argumento `parameter_device` para especificar el dispositivo en el que desea almacenar los parámetros. Por defecto utilizará la CPU, o la GPU si solo hay una.

¡Ahora veamos cómo entrenar un modelo a través de un clúster de servidores TensorFlow!


## Entrenamiento de un modelo en un clúster de TensorFlow

Un clúster de TensorFlow es un grupo de procesos de TensorFlow que se ejecutan en paralelo, generalmente en diferentes máquinas, y se comunican entre sí para completar algún trabajo (por ejemplo, entrenar o ejecutar un modelo de red neuronal). Cada proceso TF en el clúster se denomina tarea o servidor TF. Tiene una dirección IP, un puerto y un tipo (también llamado rol o trabajo). El tipo puede ser `"worker"`, `"chief"`, `"ps"` (servidor de parámetros) o `"evaluator"`:

- Cada trabajador realiza cálculos, generalmente en una máquina con una o más GPU.

* El jefe también realiza cálculos (es un trabajador), pero también maneja trabajo adicional, como escribir registros de TensorBoard o guardar puntos de control. Hay un solo jefe en un grupo. Si no se especifica explícitamente ningún jefe, entonces, por convención, el primer trabajador es el jefe.

- Un servidor de parámetros solo realiza un seguimiento de los valores variables, y por lo general está en una máquina solo de CPU. Este tipo de tarea solo se utiliza con ParameterServerStrategy.

- Obviamente, un evaluador se encarga de la evaluación. Este tipo no se usa a menudo, y cuando se usa, por lo general solo hay un evaluador.

Para iniciar un clúster de TensorFlow, primero debe definir su especificación. Esto significa definir la dirección IP, el puerto TCP y el tipo de cada tarea. Por ejemplo, la siguiente especificación de clúster define un clúster con tres tareas (dos trabajadores y un servidor de parámetros; véase la Figura 19-18). La especificación del clúster es un diccionario con una clave por trabajo, y los valores son listas de direcciones de tareas (IP:puerto):

In [ ]:
cluster_spec = {
    "worker": [
        "machine-a.example.com:2222",     # /job:worker/task:0
        "machine-b.example.com:2222"      # /job:worker/task:1
    ],
    "ps": ["machine-a.example.com:2221"]  # /job:ps/task:0
}

En general, habrá una sola tarea por máquina, pero como muestra este ejemplo, puede configurar varias tareas en la misma máquina si lo desea. En este caso, si comparten las mismas GPU, asegúrese de que la RAM se divida adecuadamente, como se discutió anteriormente.

#### ADVERTENCIA

De forma predeterminada, cada tarea en el clúster puede comunicarse con cualquier otra tarea, así que asegúrese de configurar su firewall para autorizar todas las comunicaciones entre estas máquinas en estos puertos (generalmente es más simple si utiliza el mismo puerto en todas las máquinas).

#### ---------------------------------------------------------------------------------------------------------------

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098125967/files/assets/mls3_1918.png)

(_Figura 19-18. Un ejemplo de clúster de TensorFlow_)

Cuando inicia una tarea, debe darle la especificación del clúster y también debe decirle cuál es su tipo e índice (por ejemplo, trabajador n.° 0). La forma más sencilla de especificar todo a la vez (tanto la especificación del clúster como el tipo e índice de la tarea actual) es configurar la variable de entorno TF_CONFIG antes de iniciar TensorFlow. Debe ser un diccionario codificado en JSON que contenga una especificación de clúster (bajo la clave `"cluster"`) y el tipo e índice de la tarea actual (bajo la clave `"task"`). Por ejemplo, la siguiente variable de entorno `TF_CONFIG` utiliza el clúster que acabamos de definir y especifica que la tarea a iniciar es el trabajador n.º 0:

In [ ]:
os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

#### TIP

En general, desea definir la variable de entorno `TF_CONFIG` fuera de Python, por lo que no es necesario que el código incluya el tipo y el índice de la tarea actual (esto hace posible usar el mismo código en todos los trabajadores).

#### --------------------------------------------------------------------------------------------------------------

¡Ahora entrenemos un modelo en un clúster! Comenzaremos con la estrategia reflejada. Primero, debe configurar la variable de entorno `TF_CONFIG` de forma adecuada para cada tarea. No debería haber ningún servidor de parámetros (elimine la clave `"ps"` en la especificación del clúster) y, en general, querrá un solo trabajador por máquina. Asegúrese de establecer un índice de tarea diferente para cada tarea. Finalmente, ejecute el siguiente script en cada trabajador:

In [ ]:
import tempfile
import tensorflow as tf

strategy = tf.distribute.MultiWorkerMirroredStrategy()  # at the start!
resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()
print(f"Starting task {resolver.task_type} #{resolver.task_id}")
[...] # load and split the MNIST dataset

with strategy.scope():
    model = tf.keras.Sequential([...])  # build the Keras model
    model.compile([...])  # compile the model

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10)

if resolver.task_id == 0:  # the chief saves the model to the right location
    model.save("my_mnist_multiworker_model", save_format="tf")
else:
    tmpdir = tempfile.mkdtemp()  # other workers save to a temporary directory
    model.save(tmpdir, save_format="tf")
    tf.io.gfile.rmtree(tmpdir)  # and we can delete this directory at the end!

Ese es casi el mismo código que usaste antes, excepto que esta vez estás usando `MultiWorkerMirroredStrategy`. Cuando inicies este script en los primeros trabajadores, permanecerán bloqueados en el paso AllReduce, pero la capacitación comenzará tan pronto como se inicie el último trabajador, y los verás avanzar exactamente a la misma velocidad, ya que se sincronizan en cada paso.

#### ADVERTENCIA

Al usar `MultiWorkerMirroredStrategy`, es importante asegurarse de que todos los trabajadores hagan lo mismo, incluido el almacenamiento de puntos de control del modelo o la escritura de registros de TensorBoard, a pesar de que solo mantendrá lo que el jefe escribe. Esto se debe a que estas operaciones pueden necesitar ejecutar las operaciones AllReduce, por lo que todos los trabajadores deben estar sincronizados.

#### ---------------------------------------------------------------------------------------------------------------

Hay dos implementaciones de AllReduce para esta estrategia de distribución: un algoritmo de anillo AllReduce basado en gRPC para las comunicaciones de red y la implementación de NCCL. El mejor algoritmo para usar depende del número de trabajadores, el número y los tipos de GPU y la red. De forma predeterminada, TensorFlow aplicará algunas heurísticas para seleccionar el algoritmo adecuado para usted, pero puede forzar NCCL (o RING) de esta manera:

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy(
    communication_options=tf.distribute.experimental.CommunicationOptions(
        implementation=tf.distribute.experimental.CollectiveCommunication.NCCL))

Si prefiere implementar paralelismo de datos asincrónicos con servidores de parámetros, cambie la estrategia a `ParameterServerStrategy`, agregue uno o más servidores de parámetros y configure `TF_CONFIG` adecuadamente para cada tarea. Tenga en cuenta que, aunque los trabajadores funcionarán de forma asincrónica, las réplicas de cada trabajador funcionarán de forma sincrónica.

Por último, si tiene acceso a TPU en Google Cloud (por ejemplo, si usa Colab y configura el tipo de acelerador en TPU), puede crear una TPUStrategy como esta:

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

Esto debe ejecutarse justo después de importar TensorFlow. Entonces puedes usar esta estrategia normalmente.

#### TIP

Si usted es un investigador, puede ser elegible para usar TPU de forma gratuita; consulte https://tensorflow.org/tfrc para obtener más detalles.

#### ---------------------------------------------------------------------------------------------------------------

Ahora puedes entrenar modelos a través de múltiples GPU y varios servidores: ¡date una palmadita en la espalda! Sin embargo, si quieres entrenar un modelo muy grande, necesitarás muchas GPU, en muchos servidores, lo que requerirá la compra de mucho hardware o la gestión de muchas máquinas virtuales en la nube. En muchos casos, es menos complicado y menos costoso usar un servicio en la nube que se encarga del aprovisionamiento y la gestión de toda esta infraestructura para usted, justo cuando lo necesita. Veamos cómo hacer eso usando Vertex AI.


## Ejecución de grandes trabajos de formación en Vertex AI

Vertex AI le permite crear trabajos de capacitación personalizados con su propio código de capacitación. De hecho, puedes usar casi el mismo código de entrenamiento que usarías en tu propio clúster TF. Lo principal que debes cambiar es dónde el jefe debe guardar el modelo, los puntos de control y los registros de TensorBoard. En lugar de guardar el modelo en un directorio local, el jefe debe guardarlo en GCS, utilizando la ruta proporcionada por Vertex AI en la variable de entorno `AIP_MODEL_DIR`. Para los puntos de control del modelo y los registros de TensorBoard, debe utilizar las rutas contenidas en las variables de entorno `AIP_CHECKPOINT_DIR` y `AIP_TENSORBOARD_LOG_DIR`, respectivamente. Por supuesto, también debe asegurarse de que pueda acceder a los datos de capacitación desde las máquinas virtuales, como en GCS u otro servicio de GCP como BigQuery, o directamente desde la web. Por último, Vertex AI establece explícitamente el tipo de tarea "jefe", por lo que debes identificar al jefe usando `resolve.task_type == "chief"` en lugar de `resolve.task_id == 0`:

In [ ]:
import os
[...]  # other imports, create MultiWorkerMirroredStrategy, and resolver

if resolver.task_type == "chief":
    model_dir = os.getenv("AIP_MODEL_DIR")  # paths provided by Vertex AI
    tensorboard_log_dir = os.getenv("AIP_TENSORBOARD_LOG_DIR")
    checkpoint_dir = os.getenv("AIP_CHECKPOINT_DIR")
else:
    tmp_dir = Path(tempfile.mkdtemp())  # other workers use temporary dirs
    model_dir = tmp_dir / "model"
    tensorboard_log_dir = tmp_dir / "logs"
    checkpoint_dir = tmp_dir / "ckpt"

callbacks = [tf.keras.callbacks.TensorBoard(tensorboard_log_dir),
             tf.keras.callbacks.ModelCheckpoint(checkpoint_dir)]
[...]  # build and  compile using the strategy scope, just like earlier
model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10,
          callbacks=callbacks)
model.save(model_dir, save_format="tf")

#### TIP

Si coloca los datos de entrenamiento en GCS, puede crear un `tf.data.TextLineDataset` o `tf.data.TFRecordDataset` para acceder a ellos: simplemente use las rutas de GCS como nombres de archivo (por ejemplo, gs://my_bucket/data/001.csv ). Estos conjuntos de datos dependen del paquete `tf.io.gfile` para acceder a los archivos: admite archivos locales y archivos GCS.

#### ---------------------------------------------------------------------------------------------------------------

Ahora puedes crear un trabajo de entrenamiento personalizado en Vertex AI, basado en este script. Necesitará especificar el nombre del trabajo, la ruta de su script de entrenamiento, la imagen de Docker que se utilizará para el entrenamiento, la que se utilizará para las predicciones (después del entrenamiento), cualquier biblioteca adicional de Python que pueda necesitar y, por último, el cubo que Vertex AI debe usar como directorio de ensayo para almacenar el script de entrenamiento. De forma predeterminada, ahí es también donde el script de entrenamiento guardará el modelo entrenado, así como los registros de TensorBoard y los puntos de control del modelo (si los hay). Vamos a crear el trabajo:

In [ ]:
custom_training_job = aiplatform.CustomTrainingJob(
    display_name="my_custom_training_job",
    script_path="my_vertex_ai_training_task.py",
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    model_serving_container_image_uri=server_image,
    requirements=["gcsfs==2022.3.0"],  # not needed, this is just an example
    staging_bucket=f"gs://{bucket_name}/staging"
)

Y ahora vamos a ejecutarlo en dos trabajadores, cada uno con dos GPU:

In [ ]:
mnist_model2 = custom_training_job.run(
    machine_type="n1-standard-4",
    replica_count=2,
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,
)

Y eso es todo: Vertex AI aprovisionará los nodos de computación que usted solicitó (dentro de sus cuotas) y ejecutará su script de entrenamiento en ellos. Una vez que se completa el trabajo, el método `run()` devolverá un modelo entrenado que puede usar exactamente igual al que creó anteriormente: puede implementarlo en un punto final o usarlo para hacer predicciones por lotes. Si algo sale mal durante el entrenamiento, puede ver los registros en la consola de GCP: en el menú de navegación ☰, seleccione Vertex AI → Entrenamiento, haga clic en su trabajo de entrenamiento y haga clic en VER REGISTROS. Alternativamente, puede hacer clic en la pestaña TRABAJOS PERSONALIZADOS y copiar el ID del trabajo (por ejemplo, 1234), luego seleccionar Registro en el menú de navegación ☰ y consultar `resource.labels.job_id=1234`.

#### TIP

Para visualizar el progreso del entrenamiento, simplemente inicie TensorBoard y apunte su `--logdir` a la ruta GCS de los registros. Utilizará las credenciales predeterminadas de la aplicación, que puede configurar mediante el inicio de sesión predeterminado de auth application-default login de gcloud. Vertex AI también ofrece servidores TensorBoard alojados si lo prefiere.

#### ---------------------------------------------------------------------------------------------------------------

Si desea probar algunos valores de hiperparámetros, una opción es ejecutar varios trabajos. Puede pasar los valores de los hiperparámetros a su secuencia de comandos como argumentos de la línea de comandos configurando el parámetro args al llamar al método `run()`, o puede pasarlos como variables de entorno usando el parámetro `environment_variables`.

Sin embargo, si desea ejecutar un gran trabajo de ajuste de hiperparámetros en la nube, una opción mucho mejor es utilizar el servicio de ajuste de hiperparámetros de Vertex AI. A ver cómo.


## Ajuste de hiperparámetros en Vertex AI

El servicio de ajuste de hiperparámetros de Vertex AI se basa en un algoritmo de optimización bayesiano, capaz de encontrar rápidamente combinaciones óptimas de hiperparámetros. Para usarlo, primero debe crear un script de entrenamiento que acepte valores de hiperparámetros como argumentos de la línea de comandos. Por ejemplo, su script podría usar la biblioteca estándar argparse de esta manera:

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--n_hidden", type=int, default=2)
parser.add_argument("--n_neurons", type=int, default=256)
parser.add_argument("--learning_rate", type=float, default=1e-2)
parser.add_argument("--optimizer", default="adam")
args = parser.parse_args()

El servicio de ajuste de hiperparámetros llamará a su script varias veces, cada vez con diferentes valores de hiperparámetros: cada ejecución se llama prueba, y el conjunto de pruebas se llama estudio. Su script de entrenamiento debe usar los valores de hiperparámetros dados para construir y compilar un modelo. Puede utilizar una estrategia de distribución reflejada si lo desea, en caso de que cada prueba se ejecute en una máquina multi-GPU. Entonces el script puede cargar el conjunto de datos y entrenar el modelo. Por ejemplo:

In [ ]:
import tensorflow as tf

def build_model(args):
    with tf.distribute.MirroredStrategy().scope():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=[28, 28], dtype=tf.uint8))
        for _ in range(args.n_hidden):
            model.add(tf.keras.layers.Dense(args.n_neurons, activation="relu"))
        model.add(tf.keras.layers.Dense(10, activation="softmax"))
        opt = tf.keras.optimizers.get(args.optimizer)
        opt.learning_rate = args.learning_rate
        model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
                      metrics=["accuracy"])
        return model

[...]  # load the dataset
model = build_model(args)
history = model.fit([...])

#### TIP

Puede utilizar las variables de entorno `AIP_*` que mencionamos anteriormente para determinar dónde guardar los puntos de control, los registros de TensorBoard y el modelo final.

#### ---------------------------------------------------------------------------------------------------------------

Por último, el script debe informar sobre el rendimiento del modelo al servicio de ajuste de hiperparámetros de Vertex AI, para que pueda decidir qué hiperparámetros probar a continuación. Para ello, debe utilizar la biblioteca de `hypertune`, que se instala automáticamente en las máquinas virtuales de entrenamiento de Ia de Vertex:

In [ ]:
import hypertune

hypertune = hypertune.HyperTune()
hypertune.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag="accuracy",  # name of the reported metric
    metric_value=max(history.history["val_accuracy"]),  # metric value
    global_step=model.optimizer.iterations.numpy(),
)

Ahora que su script de entrenamiento está listo, debe definir el tipo de máquina en la que le gustaría ejecutarlo. Para ello, debe definir un trabajo personalizado, que Vertex AI utilizará como plantilla para cada prueba:

In [ ]:
trial_job = aiplatform.CustomJob.from_local_script(
    display_name="my_search_trial_job",
    script_path="my_vertex_ai_trial.py",  # path to your training script
    container_uri="gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest",
    staging_bucket=f"gs://{bucket_name}/staging",
    accelerator_type="NVIDIA_TESLA_K80",
    accelerator_count=2,  # in this example, each trial will have 2 GPUs
)

Por último, estás listo para crear y ejecutar el trabajo de ajuste de hiperparámetros:

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hp_job = aiplatform.HyperparameterTuningJob(
    display_name="my_hp_search_job",
    custom_job=trial_job,
    metric_spec={"accuracy": "maximize"},
    parameter_spec={
        "learning_rate": hpt.DoubleParameterSpec(min=1e-3, max=10, scale="log"),
        "n_neurons": hpt.IntegerParameterSpec(min=1, max=300, scale="linear"),
        "n_hidden": hpt.IntegerParameterSpec(min=1, max=10, scale="linear"),
        "optimizer": hpt.CategoricalParameterSpec(["sgd", "adam"]),
    },
    max_trial_count=100,
    parallel_trial_count=20,
)
hp_job.run()

Aquí, le decimos a Vertex AI que maximice la métrica denominada `"accuracy"`: este nombre debe coincidir con el nombre de la métrica informada por el script de entrenamiento. También definimos el espacio de búsqueda, utilizando una escala logarítmica para la tasa de aprendizaje y una escala lineal (es decir, uniforme) para los otros hiperparámetros. Los nombres de los hiperparámetros deben coincidir con los argumentos de la línea de comandos del script de entrenamiento. Luego establecemos el número máximo de pruebas en 100 y el número máximo de pruebas que se ejecutan en paralelo en 20. Si aumenta el número de pruebas paralelas a (digamos) 60, el tiempo total de búsqueda se reducirá significativamente, en un factor de hasta 3. Pero los primeros 60 ensayos se iniciarán en paralelo, por lo que no se beneficiarán de la retroalimentación de los otros ensayos. Por lo tanto, debes aumentar el número máximo de pruebas para compensar, por ejemplo, hasta aproximadamente 140.

Esto llevará bastante tiempo. Una vez completado el trabajo, puede obtener los resultados de la prueba usando `hp_job.trials`. Cada resultado del ensayo se representa como un objeto protobuf, que contiene los valores de los hiperparámetros y las métricas resultantes. Vamos a encontrar la mejor prueba:

In [ ]:
def get_final_metric(trial, metric_id):
    for metric in trial.final_measurement.metrics:
        if metric.metric_id == metric_id:
            return metric.value

trials = hp_job.trials
trial_accuracies = [get_final_metric(trial, "accuracy") for trial in trials]
best_trial = trials[np.argmax(trial_accuracies)]

Ahora echemos un vistazo a la precisión de esta prueba y a sus valores de hiperparámetros:

In [ ]:
max(trial_accuracies)
#0.977400004863739

best_trial.id
#'98'

best_trial.parameters
#[parameter_id: "learning_rate" value { number_value: 0.001 },
# parameter_id: "n_hidden" value { number_value: 8.0 },
# parameter_id: "n_neurons" value { number_value: 216.0 },
# parameter_id: "optimizer" value { string_value: "adam" }
#]

¡Eso es todo! Ahora puedes obtener el SavedModel de esta prueba, entrenarlo un poco más y desplegarlo en producción.

#### TIP

Vertex AI también incluye un servicio AutoML, que se encarga completamente de encontrar la arquitectura de modelo adecuada y entrenarla para usted. Todo lo que necesita hacer es cargar su conjunto de datos en Vertex AI utilizando un formato especial que depende del tipo de conjunto de datos (imágenes, texto, tabla, vídeo, etc.), luego crear un trabajo de capacitación de AutoML, apuntando al conjunto de datos y especificando el número máximo de horas de cálculo que está dispuesto a gastar. Vea el cuaderno para ver un ejemplo.

#### ---------------------------------------------------------------------------------------------------------------

#### AJUSTE DEL HIPERPARÁMETROS USANDO EL SINTONIZADOR KERAS EN VERTEX AI

En lugar de usar el servicio de ajuste de hiperparámetros de Vertex AI, puede usar Keras Tuner (introducido en el Capítulo 10) y ejecutarlo en máquinas virtuales de Vertex AI. Keras Tuner proporciona una forma sencilla de escalar la búsqueda de hiperparámetros distribuyéndola en múltiples máquinas: solo requiere establecer tres variables de entorno en cada máquina, y luego ejecutar su código regular de Keras Tuner en cada máquina. Puedes usar exactamente el mismo script en todas las máquinas. Una de las máquinas actúa como el jefe (es decir, el oráculo) y las otras actúan como trabajadores. Cada trabajador le pregunta al jefe qué valores de hiperparámetro probar, luego el trabajador entrena el modelo utilizando estos valores de hiperparámetros y, finalmente, informa del rendimiento del modelo al jefe, que luego puede decidir qué valores de hiperparámetro debe probar el trabajador a continuación.

Las tres variables de entorno que debe establecer en cada máquina son:


`KERASTUNER_TUNER_ID`

    Esto equivale a `"chief"` en la máquina principal o a un identificador único en cada máquina trabajadora, como `"worker0"`, `"worker1"`, etc.

`KERASTUNER_ORACLE_IP`

    Esta es la dirección IP o nombre de host de la máquina principal. El propio jefe generalmente debería usar `"0.0.0.0"` para escuchar cada dirección IP en la máquina.

`KERASTUNER_ORACLE_PORT`

    Este es el puerto TCP que el jefe escuchará.

Puedes distribuir Keras Tuner en cualquier conjunto de máquinas. Si desea ejecutarlo en máquinas de inteligencia artificial de vértiza, puede crear un trabajo de entrenamiento regular y simplemente modificar el script de entrenamiento para establecer las variables de entorno correctamente antes de usar Keras Tuner. Vea el cuaderno para ver un ejemplo.

#### ---------------------------------------------------------------------------------------------------------------

Ahora tiene todas las herramientas y el conocimiento que necesita para crear arquitecturas de redes neuronales de última generación y entrenarlas a escala utilizando varias estrategias de distribución, en su propia infraestructura o en la nube, y luego implementarlas en cualquier lugar. En otras palabras, ahora tienes superpoderes: ¡úsalos bien!

# Ejercicios

1. ¿Qué contiene un SavedModel? ¿Cómo se inspecciona su contenido?

2. ¿Cuándo deberías usar TF Serving? ¿Cuáles son sus características principales? ¿Cuáles son algunas de las herramientas que puedes usar para implementarlo?

3. ¿Cómo se implementa un modelo en múltiples instancias de servicio de TF?

4. ¿Cuándo deberías usar la API gRPC en lugar de la API REST para consultar un modelo servido por TF Serving?

5. ¿Cuáles son las diferentes formas en que TFLite reduce el tamaño de un modelo para que funcione en un dispositivo móvil o integrado?

6. ¿Qué es la formación consciente de la cuantificación y por qué la necesitarías?

7. ¿Qué son el paralelismo de modelos y el paralelismo de datos? ¿Por qué se recomienda generalmente este último?

8. Al entrenar un modelo en varios servidores, ¿qué estrategias de distribución puedes usar? ¿Cómo eliges cuál usar?

9. Entrena a un modelo (cualquier modelo que te guste) e implegalo en TF Serving o Google Vertex AI. Escribe el código del cliente para consultarlo utilizando la API REST o la API gRPC. Actualice el modelo e implemente la nueva versión. Su código de cliente ahora consultará la nueva versión. Vuelve a la primera versión.

10. Train any model across multiple GPUs on the same machine using the MirroredStrategy (if you do not have access to GPUs, you can use Google Colab with a GPU runtime and create two logical GPUs). Train the model again using the CentralStorageStrategy and compare the training time.

11. Ajuste un modelo de su elección en Vertex AI, utilizando Keras Tuner o el servicio de ajuste de hiperparámetros de Vertex AI.

Las soluciones a estos ejercicios están disponibles al final del cuaderno de este capítulo, en https://homl.info/colab3.